Objetivo: configurar imports e caminhos de trabalho para leitura do arquivo em staged e escrita no diretório curated. Não há efeitos colaterais.

In [8]:
# ETAPA: PREPARO DO AMBIENTE E CAMINHOS
from pathlib import Path
import pandas as pd

# Caminhos (origem → staged; destino → curated)
PATH_STAGED  = Path(r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\data\staged\dados_operacao_consolidado_dimcorrigida.csv")
PATH_CURATED = Path(r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\data\curated\a1_physics_informed.csv")

print("Origem (staged):", PATH_STAGED)
print("Destino (curated):", PATH_CURATED)


Origem (staged): C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\data\staged\dados_operacao_consolidado_dimcorrigida.csv
Destino (curated): C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\data\curated\a1_physics_informed.csv


Objetivo: carregar o CSV de staged exatamente como está, validar a presença de status_operacao, e mostrar informações básicas (shape e primeiras colunas). Nenhuma modificação é feita ainda.

In [9]:
# ETAPA: CARGA E VALIDAÇÃO MÍNIMA DA ESTRUTURA
if not PATH_STAGED.exists():
    raise FileNotFoundError(f"Arquivo de origem não encontrado: {PATH_STAGED}")

# Leitura (sem inferências adicionais)
df_raw = pd.read_csv(PATH_STAGED, low_memory=False)

# Checagens básicas
print("Dimensões (bruto):", df_raw.shape)
print("Colunas (amostra):", list(df_raw.columns[:12]))

if "status_operacao" not in df_raw.columns:
    raise KeyError("Coluna 'status_operacao' não encontrada no arquivo de origem. Operação interrompida.")


Dimensões (bruto): (17448, 150)
Colunas (amostra): ['Timestamp', 'bed_temperature_average_adegc', 'esp_comb_with_ff_b_outl_temp_adegc', 'furnace_bed_temp_10_12_sel_adegc', 'furnace_bed_temp_13_15_sel_adegc', 'furnace_bed_temp_16_18_sel_adegc', 'furnace_bed_temp_1_3_sel_adegc', 'furnace_bed_temp_4_6_sel_adegc', 'furnace_bed_temp_7_9_sel_adegc', 'hot_reheat_steam_temperature_1_adegc', 'lms_pdr_bin_flud_air_htr_ol_te_adegc', 'miain_steam_temperature_adegc']


Objetivo: aplicar o recorte estrito status_operacao == 1, relatar quantas linhas permaneceram e a janela temporal (se houver coluna temporal chamada Timestamp). Não altera tipos ou nomes.

In [10]:
# ETAPA: RECORTE status_operacao == 1 (operação normal)
total_antes = len(df_raw)
df_cut = df_raw[df_raw["status_operacao"] == 1].copy()
total_depois = len(df_cut)

print(f"Linhas antes do recorte: {total_antes}")
print(f"Linhas após  recorte:   {total_depois}")

# Janela temporal se existir coluna temporal 'Timestamp'
ts_col = "Timestamp" if "Timestamp" in df_cut.columns else None
if ts_col:
    # Converter apenas para inspeção de janela; sem alterar o conteúdo salvo
    ts = pd.to_datetime(df_cut[ts_col], errors="coerce")
    print("Janela temporal (apenas diagnóstico):", ts.min(), "→", ts.max())
else:
    print("Coluna temporal 'Timestamp' não encontrada (diagnóstico de janela temporal omitido).")


Linhas antes do recorte: 17448
Linhas após  recorte:   11757
Janela temporal (apenas diagnóstico): 2023-03-01 07:00:00 → 2024-12-27 01:00:00


Objetivo: garantir que o diretório de destino exista e salvar o dataset já recortado, sem qualquer outra transformação, em a1_physics_informed.csv. Esta é a base mínima necessária para iniciar a linha Physics-Based.


In [11]:
# ETAPA: SALVAR DATASET FILTRADO PARA PHYSICS-BASED
PATH_CURATED.parent.mkdir(parents=True, exist_ok=True)

# Salva exatamente o que foi filtrado, sem alterar ordem de colunas
df_cut.to_csv(PATH_CURATED, index=False, encoding="utf-8-sig")

print("Arquivo salvo com sucesso em:")
print(PATH_CURATED)
print("Dimensões salvas:", df_cut.shape)


Arquivo salvo com sucesso em:
C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\data\curated\a1_physics_informed.csv
Dimensões salvas: (11757, 150)


In [12]:
from pathlib import Path

# Caminho onde será salvo o documento
output_path = Path(r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\docs\pedra_roseta_wr_wm.md")
output_path.parent.mkdir(parents=True, exist_ok=True)

# Conteúdo em Markdown
conteudo_md = """# Pedra de Roseta – Conceito Wr/Wm no Projeto A1 (Predição de Desgaste CFB)

## 1. Objetivo
Estabelecer uma linguagem e métrica únicas para avaliar e comparar taxas de desgaste de **refratário** e **metal** na caldeira CFB, aplicáveis tanto à linha *Physics-Based* quanto à linha *ML/DL*.

---

## 2. Definições Fundamentais

### Wr – *Wear rate – refractory*
Taxa de desgaste do **refratário**, representando a velocidade de perda de material ou degradação estrutural do revestimento refratário.

### Wm – *Wear rate – metal*
Taxa de desgaste do **metal**, representando a velocidade de perda de material ou degradação das partes metálicas (erosão, corrosão, abrasão).

---

## 3. Normalização por Referência

Para permitir comparabilidade e padronização, as taxas são normalizadas por referências de operação ideal:

$$
Wr_{idx} = \\frac{Wr}{Wr_{ref}}
$$

$$
Wm_{idx} = \\frac{Wm}{Wm_{ref}}
$$

Onde:

- **Wr_ref** → taxa de desgaste de referência para refratário.
- **Wm_ref** → taxa de desgaste de referência para metal.

**Benefícios da normalização:**
- Coloca variáveis em escala comparável, independente da unidade original.
- Permite análise de desvios em relação à operação projetada/ótima.
- Facilita integração com diferentes modelos e períodos operacionais.

---

## 4. Situação Atual (MVP)

- **Não há medição direta** de desgaste acumulado (mm ou kg) no momento.
- Avaliaremos **variações nas taxas** (`Wr`, `Wm`) e não o desgaste físico total.
- As referências (**Wr_ref**, **Wm_ref**) serão definidas a partir de condições estáveis de operação.
- Quando medições reais estiverem disponíveis, as referências serão recalibradas.

---

## 5. Aplicações

### Physics-Based
- Baseia-se em proxies físicos mínimos:
  - `v_proxy` → carga global (média z-score de fluxos primários).
  - `delta_proxy` → comportamento fluidodinâmico (média z-score de ΔP).
  - `tau_*` → regimes térmicos regionais (média z-score de temperaturas por região).
- Modela e calibra a relação entre condições operacionais e `Wr_idx`/`Wm_idx`.

### ML/DL
- Utiliza variáveis primárias + secundárias para treinar modelos preditivos.
- `Wr_idx` e `Wm_idx` funcionam como *targets*.
- Captura padrões complexos, mantendo alinhamento com as mesmas métricas do Physics-Based.

---

## 6. Essência do Conceito

A métrica **Wr/Wr_ref** e **Wm/Wm_ref** é a **língua comum** entre Physics-Based e ML/DL.

- Garante que ambos os fluxos compartilhem a mesma escala e objetivo.
- Permite validação cruzada: avanços em um fluxo podem ser auditados pelo outro.
- Facilita a transição do MVP (taxas relativas) para um modelo calibrado com desgaste real.

---

## 7. Quadro-Resumo

| Componente          | Símbolo       | Descrição                                               | Referência          | Métrica Normalizada |
|--------------------|--------------|-------------------------------------------------------|--------------------|--------------------|
| Desgaste Refratário | Wr           | Taxa de desgaste do refratário                        | Wr_ref (ideal)     | Wr / Wr_ref        |
| Desgaste Metal      | Wm           | Taxa de desgaste do metal                             | Wm_ref (ideal)     | Wm / Wm_ref        |

---

## 8. Observações

- `Wr_ref` e `Wm_ref` podem ser definidos por condições operacionais estáveis, históricas ou de projeto.
- A unidade de `Wr` e `Wm` (mm/ano, kg/dia, etc.) não interfere na razão, desde que Wr e Wr_ref compartilhem a mesma unidade (idem para Wm).
- O conceito será mantido e aplicado de forma idêntica nas duas linhas do projeto.
"""

# Salvar o arquivo
output_path.write_text(conteudo_md, encoding="utf-8-sig")

print(f"Documento salvo com sucesso em: {output_path}")


Documento salvo com sucesso em: C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\docs\pedra_roseta_wr_wm.md


Imports e apontar o caminho absoluto do dataset a1_physics_informed.csv já filtrado por status_operacao == 1. Validar a existência do arquivo antes de prosseguir. Não há efeitos colaterais.

In [13]:
# ETAPA 1.0 — PREPARO DO AMBIENTE E CAMINHO
from pathlib import Path
import pandas as pd
import numpy as np

PATH_PHYS = Path(r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\data\curated\a1_physics_informed.csv")

print("Arquivo alvo:", PATH_PHYS)
if not PATH_PHYS.exists():
    raise FileNotFoundError(f"Arquivo não encontrado: {PATH_PHYS}")


Arquivo alvo: C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\data\curated\a1_physics_informed.csv


Carregar o CSV sem transformar os dados; relatar dimensões, amostra das colunas e verificar a presença de status_operacao. Não grava nada.

In [14]:
# ETAPA 1.2 — DIAGNÓSTICO TEMPORAL (OPCIONAL, SE houver 'Timestamp')
ts_col = "Timestamp" if "Timestamp" in df.columns else None

if ts_col:
    df = df.sort_values(ts_col).reset_index(drop=True)
    df[ts_col] = pd.to_datetime(df[ts_col], errors="coerce")
    dups = df.duplicated(subset=[ts_col]).sum()
    deltas = df[ts_col].diff().dropna()

    print(f"Janela temporal: {df[ts_col].min()} → {df[ts_col].max()}")
    print(f"Duplicatas de {ts_col}: {dups}")
    if not deltas.empty:
        print("Top deltas (mais frequentes):")
        print(deltas.value_counts().head(5))
        print("P95 do delta temporal:", deltas.quantile(0.95))
else:
    print("Coluna temporal 'Timestamp' não encontrada — diagnóstico temporal omitido.")


Janela temporal: 2023-03-01 07:00:00 → 2024-12-27 01:00:00
Duplicatas de Timestamp: 0
Top deltas (mais frequentes):
Timestamp
0 days 01:00:00    11714
0 days 02:00:00        8
0 days 03:00:00        5
0 days 06:00:00        4
0 days 04:00:00        3
Name: count, dtype: int64
P95 do delta temporal: 0 days 01:00:00


Objetivo: checar a disponibilidade dos insumos mínimos para proxies e índices conforme a Pedra de Roseta.

     Fluxos totais (para v_proxy)

     ΔP/pressões da cama (para delta_proxy)

     Temperaturas regionais (*_adegc) (para tau_*)

     Campos de desgaste e referências (Wr, Wr_ref, Wm, Wm_ref)
Somente listar presença/ausência, exemplos e contagens de nulos.

In [15]:
# ETAPA 1.3 — PRESENÇA DE INSUMOS PARA PROXIES E ÍNDICES

# 1) Fluxos totais
fluxos = [c for c in ["flw_total_a_t_h","flw_total_b_t_h","flw_total_c_t_h"] if c in df.columns]

# 2) ΔP / pressão cama: busca por padrões auditáveis (sem criar colunas)
pads_dp = ("dp", "delta_p", "diff_press", "diff_prs", "differential", "bed_kpa", "furnace_bed_kpa")
cand_dp = [c for c in df.columns if any(p in c.lower() for p in pads_dp)]

# 3) Temperaturas regionais (adegc)
temp_cols = [c for c in df.columns if c.lower().endswith("adegc") or "_adegc" in c.lower()]

# 4) Desgaste e referências (podem não existir no MVP)
wr_col     = next((c for c in ["Wr","wr"] if c in df.columns), None)
wrref_col  = next((c for c in ["Wr_ref","wr_ref","Wrref","wrref"] if c in df.columns), None)
wm_col     = next((c for c in ["Wm","wm"] if c in df.columns), None)
wmref_col  = next((c for c in ["Wm_ref","wm_ref","Wmref","wmref"] if c in df.columns), None)

def exemplo(s):
    s_n = df[s].dropna()
    return s_n.iloc[0] if not s_n.empty else np.nan

print("— Fluxos totais:", fluxos)
for c in fluxos:
    print(f"   {c}: exemplo={exemplo(c)}, nulos={df[c].isna().sum()}")

print("\n— Candidatos ΔP/pressão cama:", len(cand_dp))
for c in cand_dp[:20]:
    print(f"   {c}: exemplo={exemplo(c)}, nulos={df[c].isna().sum()}")
if len(cand_dp) > 20:
    print(f"   ... (+{len(cand_dp)-20} colunas)")

print("\n— Temperaturas (adegc):", len(temp_cols))
for c in temp_cols:
    print(f"   {c}: exemplo={exemplo(c)}, nulos={df[c].isna().sum()}")

print("\n— Desgaste/Referências (podem estar ausentes no MVP):")
print("   Wr      :", wr_col or "—", "| Wr_ref  :", wrref_col or "—")
print("   Wm      :", wm_col or "—", "| Wm_ref  :", wmref_col or "—")


— Fluxos totais: ['flw_total_a_t_h', 'flw_total_b_t_h', 'flw_total_c_t_h']
   flw_total_a_t_h: exemplo=44.377, nulos=0
   flw_total_b_t_h: exemplo=86.946, nulos=0
   flw_total_c_t_h: exemplo=42.538, nulos=0

— Candidatos ΔP/pressão cama: 2
   pressure_1_of_furnace_b_bed_kpa: exemplo=0.899, nulos=0
   delta_proxy: exemplo=-0.5817436660816611, nulos=0

— Temperaturas (adegc): 5
   lms_pdr_bin_flud_air_htr_ol_te_adegc: exemplo=120.534, nulos=384
   te_of_hpfa_fan_outl_head_pipe_adegc: exemplo=94.024, nulos=0
   temp_of_hpfa_fan_a_outl_adegc: exemplo=95.351, nulos=1
   temp_of_hpfa_fan_b_outl_adegc: exemplo=95.351, nulos=1
   temp_of_hpfa_fan_c_outl_adegc: exemplo=23.507, nulos=0

— Desgaste/Referências (podem estar ausentes no MVP):
   Wr      : — | Wr_ref  : —
   Wm      : — | Wm_ref  : —


Garantir que estamos lendo o mesmo arquivo que você está vendo no VS Code e mostrar metadados (tamanho e data). Depois, listar todas as colunas que contêm “temp” (sem filtro por sufixo) para evidência objetiva.

In [16]:
# VERIFICAÇÃO DE ARQUIVO E INVENTÁRIO BRUTO DE TEMPERATURAS
from pathlib import Path
import pandas as pd
from datetime import datetime

PATH_PHYS = Path(r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\data\curated\a1_physics_informed.csv")
stat = PATH_PHYS.stat()
print("Arquivo:", PATH_PHYS)
print("Tamanho:", round(stat.st_size/1024,1), "KB | Modificado em:", datetime.fromtimestamp(stat.st_mtime))

df = pd.read_csv(PATH_PHYS, low_memory=False)

# inventário bruto (qualquer coisa com 'temp' no nome)
temp_like = [c for c in df.columns if "temp" in c.lower()]
print(f"Colunas contendo 'temp': {len(temp_like)}")
for c in temp_like[:60]:
    print(" -", c)
if len(temp_like) > 60:
    print(f"... (+{len(temp_like)-60} outras)")


Arquivo: C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\data\curated\a1_physics_informed.csv
Tamanho: 12456.6 KB | Modificado em: 2025-08-11 09:33:05.744416
Colunas contendo 'temp': 35
 - bed_temperature_average_adegc
 - esp_comb_with_ff_b_outl_temp_adegc
 - furnace_bed_temp_10_12_sel_adegc
 - furnace_bed_temp_13_15_sel_adegc
 - furnace_bed_temp_16_18_sel_adegc
 - furnace_bed_temp_1_3_sel_adegc
 - furnace_bed_temp_4_6_sel_adegc
 - furnace_bed_temp_7_9_sel_adegc
 - hot_reheat_steam_temperature_1_adegc
 - miain_steam_temperature_adegc
 - t_bfbp_b_inlet_temperature_adegc
 - temp_1_of_furnace_a_inner_adegc
 - temp_1_of_furnace_b_inner_adegc
 - temp_3_of_furnace_a_inner_adegc
 - temp_3_of_furnace_b_inner_adegc
 - temp_of_3_lph_outl_wtr_adegc
 - temp_of_4_lph_outl_wtr_adegc
 - temp_of_cnd_pump_outl_header_adegc
 - temp_of_cnd_water_of_dea_inl_adegc
 - temp_of_furnace_a_outl_adegc
 - temp_of_furnace_b_outl_adegc
 - temp_of_gas_in_aph_outl_adegc
 - temp_of_gas_in_ltr_side_a_inl_adegc
 - tem

plano de verificação (ultra v2)

     carregar a1_physics_informed.csv (apenas leitura).

     detectar colunas de temperatura com normalização forte de nomes.

     classificar cada coluna em grupos físicos por padrões auditáveis.

     coletar exemplo de valor e contagem de nulos.

     salvar em outputs/inventario_temperaturas.csv.

 importar libs, definir caminhos e funções de normalização (remover acentos, espaços, símbolos, padronizar para comparação). não há efeitos colaterais.

In [17]:
# ETAPA 0 — PREPARO
from pathlib import Path
import pandas as pd
import numpy as np
import re
import unicodedata
from datetime import datetime

# caminhos
PATH_PHYS = Path(r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\data\curated\a1_physics_informed.csv")
PATH_OUT  = Path(r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\outputs\inventario_temperaturas.csv")

# util — normalização forte para comparação de nomes
def norm(s: str) -> str:
    s = "".join(ch for ch in unicodedata.normalize("NFKD", s) if not unicodedata.combining(ch))
    s = s.strip().lower()
    s = re.sub(r"\s+", "_", s)
    s = s.replace("-", "_")
    s = re.sub(r"[^a-z0-9_]", "", s)
    s = re.sub(r"_+", "_", s).strip("_")
    return s


ler a base e inventariar todas as colunas de temperatura, considerando variações (*_temp_*, *_temperature_*, *_adeg c, *_degc, etc.). mostramos contagem e alguns exemplos em tela para auditoria. não grava nada ainda.

In [18]:
# ETAPA 1 — LEITURA E DETECÇÃO ROBUSTA DE TEMPERATURAS
if not PATH_PHYS.exists():
    raise FileNotFoundError(f"Arquivo não encontrado: {PATH_PHYS}")

df = pd.read_csv(PATH_PHYS, low_memory=False)
cols_norm = {c: norm(c) for c in df.columns}

def is_temperature(nc: str) -> bool:
    # qualquer coluna que mencione 'temp' ou 'temperature' ou termine em degc/adeg
    if "temp" in nc or "temperature" in nc:
        return True
    if nc.endswith("degc") or nc.endswith("adeg"):
        return True
    # casos como *_temp_adegc, *_temperature_adegc etc.
    if re.search(r"(?:_|^)(?:temp|temperature)_(?:.*_)?(?:degc|adeg)(?:_|$)", nc):
        return True
    return False

temp_cols = [orig for orig, nc in cols_norm.items() if is_temperature(nc)]
print(f"Temperaturas detectadas: {len(temp_cols)}")
for c in sorted(temp_cols)[:25]:
    print(" -", c)
if len(temp_cols) > 25:
    print(f"... (+{len(temp_cols)-25} outras)")


Temperaturas detectadas: 43
 - bed_temperature_average_adegc
 - esp_comb_with_ff_b_outl_temp_adegc
 - fsh_outl_hdr_temp_adegc_min
 - furnace_bed_temp_10_12_sel_adegc
 - furnace_bed_temp_13_15_sel_adegc
 - furnace_bed_temp_16_18_sel_adegc
 - furnace_bed_temp_1_3_sel_adegc
 - furnace_bed_temp_4_6_sel_adegc
 - furnace_bed_temp_7_9_sel_adegc
 - hi_te_rh_ol_temp_adegc_min
 - hi_temperture_reheol_pres_mpa_mi
 - hot_reheat_steam_temperature_1_adegc
 - lms_pdr_bin_flud_air_htr_ol_te_adegc
 - miain_steam_temperature_adegc
 - t_bfbp_b_inlet_temperature_adegc
 - te_no_6_hp_heater_ol_feed_wtr_adegc
 - te_no_7_hp_heater_ol_feed_wtr_adegc
 - te_no_8_hp_heater_ol_feed_wtr_adegc
 - te_of_gas_in_ecmz_side_a_outl_adegc
 - te_of_gas_in_ecmz_side_b_outl_adegc
 - te_of_hot_pri_air_in_aph_outl_adegc
 - te_of_hpfa_fan_outl_head_pipe_adegc
 - temp_1_of_furnace_a_inner_adegc
 - temp_1_of_furnace_b_inner_adegc
 - temp_3_of_furnace_a_inner_adegc
... (+18 outras)


classificar cada temperatura em grupo físico por padrões de nome. as regras são transparentes e fáceis de ajustar. nenhuma escrita ainda.

In [19]:
# ETAPA 2 — CLASSIFICAÇÃO EM GRUPOS FÍSICOS (REGRAS AUDITÁVEIS)

GROUP_RULES = [
    ("bed",          [r"\bbed\b", r"furnace_bed", r"\bbfb\b", r"\bfreeboard\b"]),   # cama / áreas de leito
    ("cyclone",      [r"\bcyclone\b"]),
    ("fan",          [r"\bhpfa\b", r"\bsa_fan\b", r"\bfan\b"]),                     # ventiladores/HPFA/SA
    ("gas_path",     [r"\baph\b", r"\bltr\b", r"\bgas_in\b", r"\bgas_out\b"]),
    ("steam",        [r"\bmain_steam\b", r"\breheat\b", r"\bhot_reheat\b", r"\bfs?h_outl_hdr\b"]),
    ("water",        [r"\bdea\b", r"\bcnd\b", r"\bcondensate\b", r"\blph\b", r"\bfeedwater\b", r"\bwtr\b"]),
    ("oil",          [r"\boil\b", r"\bolio\b"]),
    ("furnace",      [r"\bfurnace\b"]),  # genérico de fornalha (não-bed)
    ("others",       [r".*"]),           # fallback
]

def assign_group(nc: str) -> str:
    for group, patterns in GROUP_RULES:
        for pat in patterns:
            if re.search(pat, nc):
                return group
    return "others"

# tabela base (ainda sem salvar)
rows = []
for col in temp_cols:
    nc = cols_norm[col]
    grp = assign_group(nc)
    s = pd.to_numeric(df[col], errors="coerce")
    exemplo = s.dropna().iloc[0] if s.notna().any() else np.nan
    rows.append({
        "variavel": col,
        "variavel_normalizada": nc,
        "grupo_fisico": grp,
        "exemplo_valor": exemplo,
        "nulos": int(s.isna().sum()),
    })

inv = pd.DataFrame(rows)
# ordenação amigável: grupo, nome normalizado
inv = inv.sort_values(["grupo_fisico", "variavel_normalizada"]).reset_index(drop=True)

print("Prévia (top 15 linhas):")
inv.head(15)


Prévia (top 15 linhas):


,variavel,variavel_normalizada,grupo_fisico,exemplo_valor,nulos
0,furnace_bed_temp_10_12_sel_adegc,furnace_bed_temp_10_12_sel_adegc,bed,760.133,0
1,furnace_bed_temp_13_15_sel_adegc,furnace_bed_temp_13_15_sel_adegc,bed,753.804,0
2,furnace_bed_temp_16_18_sel_adegc,furnace_bed_temp_16_18_sel_adegc,bed,780.632,0
3,furnace_bed_temp_1_3_sel_adegc,furnace_bed_temp_1_3_sel_adegc,bed,732.287,0
4,furnace_bed_temp_4_6_sel_adegc,furnace_bed_temp_4_6_sel_adegc,bed,737.218,0
5,furnace_bed_temp_7_9_sel_adegc,furnace_bed_temp_7_9_sel_adegc,bed,763.715,0
6,bed_temperature_average_adegc,bed_temperature_average_adegc,others,761.795,2723
7,esp_comb_with_ff_b_outl_temp_adegc,esp_comb_with_ff_b_outl_temp_adegc,others,101.267,1
8,fsh_outl_hdr_temp_adegc_min,fsh_outl_hdr_temp_adegc_min,others,-0.358,0
9,hi_te_rh_ol_temp_adegc_min,hi_te_rh_ol_temp_adegc_min,others,-0.358,0


objetivo: salvar o inventário de temperaturas com grupo físico, exemplo e nulos para uso em todo o fluxo physics_based. cria a pasta outputs se não existir.

In [20]:
# ETAPA 3 — SALVAR INVENTÁRIO EM CSV
PATH_OUT.parent.mkdir(parents=True, exist_ok=True)
inv.to_csv(PATH_OUT, index=False, encoding="utf-8-sig")

print("Inventário salvo em:", PATH_OUT)
print("Total de variáveis de temperatura inventariadas:", len(inv))


Inventário salvo em: C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\outputs\inventario_temperaturas.csv
Total de variáveis de temperatura inventariadas: 43


Plano de verificação
     Ler a1_physics_informed.csv (somente leitura).

     Detectar colunas de temperatura com normalização forte de nomes.

     Criar uma coluna nova: equipamento_processo, usando:

     Overrides explícitos para os nomes que você descreveu.

     Regras auditáveis (regex) como fallback.

     Manter grupo_fisico (como antes).

     Regerar o inventário com: variavel, variavel_normalizada, grupo_fisico, equipamento_processo, exemplo_valor, nulos.

     Salvar sobrescrevendo outputs\inventario_temperaturas.csv.

     Exibir um resumo por grupo e por equipamento/processo (somente em tela

In [21]:
# ETAPA 0 — PREPARO
from pathlib import Path
import pandas as pd
import numpy as np
import re
import unicodedata

PATH_PHYS = Path(r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\data\curated\a1_physics_informed.csv")
PATH_OUT  = Path(r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\outputs\inventario_temperaturas.csv")

def norm(s: str) -> str:
    s = "".join(ch for ch in unicodedata.normalize("NFKD", s) if not unicodedata.combining(ch))
    s = s.strip().lower()
    s = re.sub(r"\s+", "_", s)           # espaços → _
    s = s.replace("-", "_")
    s = re.sub(r"[^a-z0-9_]", "", s)     # apenas a-z0-9_
    s = re.sub(r"_+", "_", s).strip("_") # colapsa múltiplos _
    return s


Objetivo: ler a base, detectar todas as colunas de temperatura (robusto a variações) e construir visão normalizada (sem alterar o DataFrame original em disco).

In [22]:
# ETAPA 1 — LEITURA E DETECÇÃO ROBUSTA DE TEMPERATURAS
if not PATH_PHYS.exists():
    raise FileNotFoundError(f"Arquivo não encontrado: {PATH_PHYS}")

df = pd.read_csv(PATH_PHYS, low_memory=False)
cols_norm = {c: norm(c) for c in df.columns}

def is_temperature(nc: str) -> bool:
    # Marca como temperatura se contém temp/temperature ou termina em degc/adeg
    if "temp" in nc or "temperature" in nc:
        return True
    if nc.endswith("degc") or nc.endswith("adeg"):
        return True
    # *_temp_adegc / *_temperature_adegc / variações
    if re.search(r"(?:_|^)(?:temp|temperature)_(?:.*_)?(?:degc|adeg)(?:_|$)", nc):
        return True
    return False

temp_cols = [orig for orig, nc in cols_norm.items() if is_temperature(nc)]
print(f"Temperaturas detectadas: {len(temp_cols)}")


Temperaturas detectadas: 43


In [24]:
# ETAPA 2A — OVERRIDES (EQUIPAMENTO/PROCESSO) A PARTIR DO SEU MAPEAMENTO
overrides = {
    # Leito/Fornalha – bed/furnace bed
    "bed_temperature_average_adegc": "Fornalha – Leito (média)",
    "furnace_bed_temp_1_3_sel_adegc":  "Fornalha – Leito (segmento 1–3)",
    "furnace_bed_temp_4_6_sel_adegc":  "Fornalha – Leito (segmento 4–6)",
    "furnace_bed_temp_7_9_sel_adegc":  "Fornalha – Leito (segmento 7–9)",
    "furnace_bed_temp_10_12_sel_adegc":"Fornalha – Leito (segmento 10–12)",
    "furnace_bed_temp_13_15_sel_adegc":"Fornalha – Leito (segmento 13–15)",
    "furnace_bed_temp_16_18_sel_adegc":"Fornalha – Leito (segmento 16–18)",
    "t_bfbp_b_inlet_temperature_adegc":"Fornalha – Leito inferior (seção B, entrada)",
    "temp_1_of_furnace_a_inner_adegc": "Fornalha A – Interna (ponto 1)",
    "temp_3_of_furnace_a_inner_adegc": "Fornalha A – Interna (ponto 3)",
    "temp_1_of_furnace_b_inner_adegc": "Fornalha B – Interna (ponto 1)",
    "temp_3_of_furnace_b_inner_adegc": "Fornalha B – Interna (ponto 3)",
    "temp_of_furnace_a_outl_adegc":    "Fornalha A – Saída de gases",
    "temp_of_furnace_b_outl_adegc":    "Fornalha B – Saída de gases",

    # Ciclones
    "temperature_of_cyclone_a_inlet_adegc": "Ciclone A – Entrada",
    "temperature_of_cyclone_b_inlet_adegc": "Ciclone B – Entrada",
    "temperature_of_cyclone_c_inlet_adegc": "Ciclone C – Entrada",

    # Gases / APH / LTR / Economizer
    "temp_of_gas_in_aph_outl_adegc":        "Gases – APH – Saída",
    "te_of_hot_pri_air_in_aph_outl_adegc":  "Ar de combustão – APH – Ar primário quente – Saída",
    "temp_of_gas_in_ltr_side_a_inl_adegc":  "Gases – LTR lado A – Entrada",
    "temp_of_gas_in_ltr_side_a_outl_adegc": "Gases – LTR lado A – Saída",
    "temp_of_gas_in_ltr_side_b_inl_adegc":  "Gases – LTR lado B – Entrada",
    "temp_of_gas_in_ltr_side_b_outl_adegc": "Gases – LTR lado B – Saída",
    "te_of_gas_in_ecmz_side_a_outl_adegc":  "Gases – Economizer (ECMZ) lado A – Saída",
    "te_of_gas_in_ecmz_side_b_outl_adegc":  "Gases – Economizer (ECMZ) lado B – Saída",

    # Fans / HPFA
    "te_of_hpfa_fan_outl_head_pipe_adegc":  "HPFA – Tubo de cabeçote – Saída",
    "temp_of_hpfa_fan_a_outl_adegc":        "HPFA Fan A – Saída",
    "temp_of_hpfa_fan_b_outl_adegc":        "HPFA Fan B – Saída",
    "temp_of_hpfa_fan_c_outl_adegc":        "HPFA Fan C – Saída",

    # Vapor
    "miain_steam_temperature_adegc":        "Vapor principal – Temperatura (verificar: main_steam)",
    "main_steam_temperature_adegc":         "Vapor principal – Temperatura",
    "hot_reheat_steam_temperature_1_adegc": "Vapor – Reheat quente (ponto 1)",
    "fsh_outl_hdr_temp_adegc_min":          "Vapor – Superaquecedor final – Cabeçote de saída (mín)",
    "hi_te_rh_ol_temp_adegc_min":           "Vapor – Reaquecedor alta temperatura – Saída (mín)",

    # Água / Condensado / LPH / DEA
    "temp_of_cnd_pump_outl_header_adegc":   "Ciclo água – Condensado – Bomba (coletor de saída)",
    "temp_of_cnd_water_of_dea_inl_adegc":   "Ciclo água – Condensado – Entrada do DEA",
    "temp_of_3_lph_outl_wtr_adegc":         "Ciclo água – LPH #3 – Saída de água",
    "temp_of_4_lph_outl_wtr_adegc":         "Ciclo água – LPH #4 – Saída de água",
    "te_no_6_hp_heater_ol_feed_wtr_adegc":  "Ciclo água – HP Heater #6 – Saída água de alimentação",
    "te_no_7_hp_heater_ol_feed_wtr_adegc":  "Ciclo água – HP Heater #7 – Saída água de alimentação",
    "te_no_8_hp_heater_ol_feed_wtr_adegc":  "Ciclo água – HP Heater #8 – Saída água de alimentação",

    # Materiais (Calcário) / Ar de fluidização
    "lms_pdr_bin_flud_air_htr_ol_te_adegc": "Manuseio de calcário – Aquecedor de ar de fluidização (silo) – Saída",

    # Emissões (ESP/FF)
    "esp_comb_with_ff_b_outl_temp_adegc":   "Controle de emissões – ESP/FF lado B – Saída",
}

# Fallback: regras auditáveis (regex) → equipamento_processo
def equipamento_from_patterns(nc: str) -> str:
    if re.search(r"furnace_bed|(^|_)bed($|_)", nc): return "Fornalha – Leito (bed)"
    if re.search(r"furnace_.*_outl", nc):           return "Fornalha – Saída de gases"
    if re.search(r"furnace_.*_inner", nc):          return "Fornalha – Interna"
    if "cyclone" in nc and "inlet" in nc:           return "Ciclone – Entrada"
    if "aph" in nc and "gas" in nc:                 return "Gases – APH"
    if "ltr" in nc and "gas" in nc:                 return "Gases – LTR"
    if "ecmz" in nc or "econom" in nc:              return "Gases – Economizer"
    if "hpfa" in nc and "fan" in nc:                return "HPFA – Fan/Tubulação"
    if "reheat" in nc or "rh" in nc:                return "Vapor – Reheat"
    if "main_steam" in nc or "miain_steam" in nc:   return "Vapor – Principal"
    if any(x in nc for x in ["dea","cnd","condens","lph","feed_wtr","wtr"]): return "Ciclo água"
    if any(x in nc for x in ["esp","ff"]):          return "Controle de emissões – ESP/FF"
    if any(x in nc for x in ["oil","ol_"]):         return "Aquecedor/óleo"
    if "gas" in nc:                                 return "Caminho de gases"
    return "Outros"


 classificar cada coluna de temperatura em grupo_fisico (macro) e na nova coluna equipamento_processo (fina), gerar inventário completo e salvar.

In [25]:
# ETAPA 2B — CLASSIFICAÇÃO FINAL E INVENTÁRIO
GROUP_RULES = [
    ("bed",          [r"\bbed\b", r"furnace_bed", r"\bfreeboard\b", r"\bbfb\b"]),
    ("cyclone",      [r"\bcyclone\b"]),
    ("fan",          [r"\bhpfa\b", r"\bsa_fan\b", r"\bfan\b"]),
    ("gas_path",     [r"\baph\b", r"\bltr\b", r"\bgas_in\b", r"\bgas_out\b", r"\becmz\b", r"\beconom"]),
    ("steam",        [r"\bmain_steam\b", r"\breheat\b", r"\bhot_reheat\b", r"\bfsh\b"]),
    ("water",        [r"\bdea\b", r"\bcnd\b", r"\bcondens", r"\blph\b", r"\bfeed_wtr\b", r"\bwtr\b"]),
    ("emissions",    [r"\besp\b", r"\bff\b"]),
    ("materials",    [r"\blms\b", r"\blimestone\b", r"\bflud_air_htr\b"]),
    ("furnace",      [r"\bfurnace\b"]),  # genérico de fornalha (não-bed)
    ("others",       [r".*"]),           # fallback
]

def assign_group(nc: str) -> str:
    for group, patterns in GROUP_RULES:
        for pat in patterns:
            if re.search(pat, nc):
                return group
    return "others"

rows = []
for col in temp_cols:
    nc = cols_norm[col]
    grupo = assign_group(nc)
    # prioridade: override → fallback por padrões
    equip = overrides.get(nc, equipamento_from_patterns(nc))
    s = pd.to_numeric(df[col], errors="coerce")
    exemplo = s.dropna().iloc[0] if s.notna().any() else np.nan
    rows.append({
        "variavel": col,
        "variavel_normalizada": nc,
        "grupo_fisico": grupo,
        "equipamento_processo": equip,     # ← NOVA COLUNA
        "exemplo_valor": exemplo,
        "nulos": int(s.isna().sum()),
    })

inv = pd.DataFrame(rows).sort_values(
    ["grupo_fisico", "equipamento_processo", "variavel_normalizada"]
).reset_index(drop=True)

# Salvar (sobrescreve o arquivo anterior)
PATH_OUT.parent.mkdir(parents=True, exist_ok=True)
inv.to_csv(PATH_OUT, index=False, encoding="utf-8-sig")

print("Inventário salvo em:", PATH_OUT)
print("Total de variáveis inventariadas:", len(inv))
display(inv.head(12))


Inventário salvo em: C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\outputs\inventario_temperaturas.csv
Total de variáveis inventariadas: 43


,variavel,variavel_normalizada,grupo_fisico,equipamento_processo,exemplo_valor,nulos
0,furnace_bed_temp_10_12_sel_adegc,furnace_bed_temp_10_12_sel_adegc,bed,Fornalha – Leito (segmento 10–12),760.133,0
1,furnace_bed_temp_13_15_sel_adegc,furnace_bed_temp_13_15_sel_adegc,bed,Fornalha – Leito (segmento 13–15),753.804,0
2,furnace_bed_temp_16_18_sel_adegc,furnace_bed_temp_16_18_sel_adegc,bed,Fornalha – Leito (segmento 16–18),780.632,0
3,furnace_bed_temp_1_3_sel_adegc,furnace_bed_temp_1_3_sel_adegc,bed,Fornalha – Leito (segmento 1–3),732.287,0
4,furnace_bed_temp_4_6_sel_adegc,furnace_bed_temp_4_6_sel_adegc,bed,Fornalha – Leito (segmento 4–6),737.218,0
5,furnace_bed_temp_7_9_sel_adegc,furnace_bed_temp_7_9_sel_adegc,bed,Fornalha – Leito (segmento 7–9),763.715,0
6,hi_temperture_reheol_pres_mpa_mi,hi_temperture_reheol_pres_mpa_mi,others,Aquecedor/óleo,0.006,0
7,te_of_hot_pri_air_in_aph_outl_adegc,te_of_hot_pri_air_in_aph_outl_adegc,others,Ar de combustão – APH – Ar primário quente – S...,261.133,0
8,temp_of_cnd_pump_outl_header_adegc,temp_of_cnd_pump_outl_header_adegc,others,Ciclo água – Condensado – Bomba (coletor de sa...,45.459,0
9,temp_of_cnd_water_of_dea_inl_adegc,temp_of_cnd_water_of_dea_inl_adegc,others,Ciclo água – Condensado – Entrada do DEA,69.035,0


Objetivo: refazer a avaliação: sumários em tela por macrogrupo e por equipamento/processo (top 20), para você validar rapidamente se o mapeamento ficou correto.

In [26]:
# ETAPA 3 — AVALIAÇÃO/RESUMO (EM TELA, NÃO SALVA)
print("\nResumo por grupo_fisico (contagem):")
display(inv.groupby("grupo_fisico")["variavel"].count().sort_values(ascending=False).to_frame("qtde"))

print("\nTop 20 — equipamento_processo (contagem):")
display(inv.groupby("equipamento_processo")["variavel"].count().sort_values(ascending=False).head(20).to_frame("qtde"))



Resumo por grupo_fisico (contagem):


,qtde
grupo_fisico,
others,37
bed,6



Top 20 — equipamento_processo (contagem):


,qtde
equipamento_processo,
Aquecedor/óleo,1
Ar de combustão – APH – Ar primário quente – Saída,1
Ciclo água – Condensado – Bomba (coletor de saída),1
Ciclo água – Condensado – Entrada do DEA,1
Ciclo água – HP Heater #6 – Saída água de alimentação,1
Ciclo água – HP Heater #7 – Saída água de alimentação,1
Ciclo água – HP Heater #8 – Saída água de alimentação,1
Ciclo água – LPH #3 – Saída de água,1
Ciclo água – LPH #4 – Saída de água,1


Carregar a base, reaproveitar os overrides que você aprovou, reforçar o léxico e classificar com prioridade rígida (sem cair em “others”). Não grava nada.

In [31]:
# ETAPA A — RECLASSIFICAÇÃO DETERMINÍSTICA (LÉXICO + REGEX) E DERIVAÇÃO DE 'tau_zona'
from pathlib import Path
import pandas as pd
import numpy as np
import re, unicodedata

PATH_PHYS = Path(r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\data\curated\a1_physics_informed.csv")
df = pd.read_csv(PATH_PHYS, low_memory=False)

def norm(s: str) -> str:
    s = "".join(ch for ch in unicodedata.normalize("NFKD", s) if not unicodedata.combining(ch))
    s = re.sub(r"\s+", "_", s.strip().lower())
    s = s.replace("-", "_")
    s = re.sub(r"[^a-z0-9_]", "", s)
    return re.sub(r"_+", "_", s).strip("_")

cols_norm = {c: norm(c) for c in df.columns}

def is_temperature(nc: str) -> bool:
    if "temp" in nc or "temperature" in nc: return True
    if nc.endswith("degc") or nc.endswith("adeg"): return True
    return bool(re.search(r"(?:_|^)(?:temp|temperature)_(?:.*_)?(?:degc|adeg)(?:_|$)", nc))

temp_cols = [orig for orig, nc in cols_norm.items() if is_temperature(nc)]

# 1) overrides (equipamento/processo) já validados previamente
overrides = {
    "bed_temperature_average_adegc": "Fornalha – Leito (média)",
    "furnace_bed_temp_1_3_sel_adegc":  "Fornalha – Leito (segmento 1–3)",
    "furnace_bed_temp_4_6_sel_adegc":  "Fornalha – Leito (segmento 4–6)",
    "furnace_bed_temp_7_9_sel_adegc":  "Fornalha – Leito (segmento 7–9)",
    "furnace_bed_temp_10_12_sel_adegc":"Fornalha – Leito (segmento 10–12)",
    "furnace_bed_temp_13_15_sel_adegc":"Fornalha – Leito (segmento 13–15)",
    "furnace_bed_temp_16_18_sel_adegc":"Fornalha – Leito (segmento 16–18)",
    "t_bfbp_b_inlet_temperature_adegc":"Fornalha – Leito inferior (seção B, entrada)",
    "temp_1_of_furnace_a_inner_adegc": "Fornalha A – Interna (ponto 1)",
    "temp_3_of_furnace_a_inner_adegc": "Fornalha A – Interna (ponto 3)",
    "temp_1_of_furnace_b_inner_adegc": "Fornalha B – Interna (ponto 1)",
    "temp_3_of_furnace_b_inner_adegc": "Fornalha B – Interna (ponto 3)",
    "temp_of_furnace_a_outl_adegc":    "Fornalha A – Saída de gases",
    "temp_of_furnace_b_outl_adegc":    "Fornalha B – Saída de gases",
    "temperature_of_cyclone_a_inlet_adegc": "Ciclone A – Entrada",
    "temperature_of_cyclone_b_inlet_adegc": "Ciclone B – Entrada",
    "temperature_of_cyclone_c_inlet_adegc": "Ciclone C – Entrada",
    "temp_of_gas_in_aph_outl_adegc":        "Gases – APH – Saída",
    "te_of_hot_pri_air_in_aph_outl_adegc":  "Ar de combustão – APH – Ar primário quente – Saída",
    "temp_of_gas_in_ltr_side_a_inl_adegc":  "Gases – LTR lado A – Entrada",
    "temp_of_gas_in_ltr_side_a_outl_adegc": "Gases – LTR lado A – Saída",
    "temp_of_gas_in_ltr_side_b_inl_adegc":  "Gases – LTR lado B – Entrada",
    "temp_of_gas_in_ltr_side_b_outl_adegc": "Gases – LTR lado B – Saída",
    "te_of_gas_in_ecmz_side_a_outl_adegc":  "Gases – Economizer lado A – Saída",
    "te_of_gas_in_ecmz_side_b_outl_adegc":  "Gases – Economizer lado B – Saída",
    "te_of_hpfa_fan_outl_head_pipe_adegc":  "HPFA – Tubo de cabeçote – Saída",
    "temp_of_hpfa_fan_a_outl_adegc":        "HPFA Fan A – Saída",
    "temp_of_hpfa_fan_b_outl_adegc":        "HPFA Fan B – Saída",
    "temp_of_hpfa_fan_c_outl_adegc":        "HPFA Fan C – Saída",
    "miain_steam_temperature_adegc":        "Vapor principal – Temperatura (verificar: main_steam)",
    "main_steam_temperature_adegc":         "Vapor principal – Temperatura",
    "hot_reheat_steam_temperature_1_adegc": "Vapor – Reheat quente (ponto 1)",
    "fsh_outl_hdr_temp_adegc_min":          "Vapor – Superaquecedor final – Cabeçote de saída (mín)",
    "hi_te_rh_ol_temp_adegc_min":           "Vapor – Reaquecedor alta temperatura – Saída (mín)",
    "temp_of_cnd_pump_outl_header_adegc":   "Ciclo água – Condensado – Bomba (coletor de saída)",
    "temp_of_cnd_water_of_dea_inl_adegc":   "Ciclo água – Condensado – Entrada do DEA",
    "temp_of_3_lph_outl_wtr_adegc":         "Ciclo água – LPH #3 – Saída de água",
    "temp_of_4_lph_outl_wtr_adegc":         "Ciclo água – LPH #4 – Saída de água",
    "te_no_6_hp_heater_ol_feed_wtr_adegc":  "Ciclo água – HP Heater #6 – Saída água de alimentação",
    "te_no_7_hp_heater_ol_feed_wtr_adegc":  "Ciclo água – HP Heater #7 – Saída água de alimentação",
    "te_no_8_hp_heater_ol_feed_wtr_adegc":  "Ciclo água – HP Heater #8 – Saída água de alimentação",
    "lms_pdr_bin_flud_air_htr_ol_te_adegc": "Calcário – Aquecedor ar de fluidização do silo – Saída",
    "esp_comb_with_ff_b_outl_temp_adegc":   "Emissões – ESP/FF lado B – Saída",
}

# 2) macro-grupo 'grupo_fisico' (para validação geral) — prioridade
GROUP_RULES = [
    ("bed",      [r"(?:^|_)bed(?:_|$)", r"furnace_bed", r"\bfreeboard\b", r"\bbfb\b"]),
    ("cyclone",  [r"\bcyclone\b"]),
    ("fan",      [r"\bhpfa\b", r"\bsa_fan\b", r"(?:^|_)fan(?:_|$)"]),
    ("gas_path", [r"\baph\b", r"\bltr\b", r"\becmz\b", r"econom", r"(?:^|_)gas(?:_|$)"]),
    ("steam",    [r"main_steam", r"reheat", r"\bfsh\b", r"\bhot_reheat\b"]),
    ("water",    [r"\bdea\b", r"\bcnd\b", r"condens", r"\blph\b", r"feed_wtr", r"(?:^|_)wtr(?:_|$)"]),
    ("emissions",[r"(?:^|_)esp(?:_|$)", r"(?:^|_)ff(?:_|$)"]),
    ("materials",[r"\blms\b", r"limestone", r"flud_air_htr"]),
    ("furnace",  [r"\bfurnace\b"]),  # genérico
]

def assign_group(nc: str) -> str:
    for group, pats in GROUP_RULES:
        for pat in pats:
            if re.search(pat, nc):
                return group
    return "others"

# 3) derivar 'equipamento_processo' (override → fallback)
def equipamento_from_patterns(nc: str) -> str:
    if re.search(r"furnace_bed|(^|_)bed($|_)", nc): return "Fornalha – Leito"
    if re.search(r"furnace_.*_inner", nc):          return "Fornalha – Interna"
    if re.search(r"furnace_.*_outl", nc):           return "Fornalha – Saída de gases"
    if "cyclone" in nc and "inlet" in nc:           return "Ciclone – Entrada"
    if "aph" in nc and "gas" in nc:                 return "Gases – APH"
    if "ltr" in nc and "gas" in nc:                 return "Gases – LTR"
    if "ecmz" in nc or "econom" in nc:              return "Gases – Economizer"
    if "hpfa" in nc and "fan" in nc:                return "HPFA – Fan/Tubo"
    if "reheat" in nc or "rh" in nc:                return "Vapor – Reheat"
    if "main_steam" in nc or "miain_steam" in nc:   return "Vapor – Principal"
    if any(x in nc for x in ["dea","cnd","condens","lph","feed_wtr","wtr"]): return "Ciclo água"
    if any(x in nc for x in ["esp","ff"]):          return "Emissões – ESP/FF"
    if any(x in nc for x in ["oil","ol_"]):         return "Aquecimento/óleo"
    if "gas" in nc:                                 return "Caminho de gases"
    return "Outros"

# 4) derivar 'tau_zona' para redução de cardinalidade
def tau_zona_from(equip: str, grp: str, nc: str) -> str:
    # ordem de prioridade sem ambiguidade:
    if "Leito" in equip or grp == "bed":         return "densa"
    if ("Fornalha – Interna" in equip) or ("Fornalha – Saída" in equip): return "diluida"
    if grp in ["gas_path","cyclone"] or any(k in equip for k in ["APH","LTR","Economizer","Ciclone"]): return "backpass"
    if grp == "fan" or "HPFA" in equip or "óleo" in equip or "Aquecimento" in equip: return "aux_maq"
    if grp == "steam" or "Vapor" in equip:       return "ciclo_vapor"
    if grp == "water" or "Ciclo água" in equip:  return "ciclo_agua"
    if grp == "emissions" or "Emissões" in equip:return "emissoes"
    return "indefinido"

rows = []
for col in temp_cols:
    nc = cols_norm[col]
    grp = assign_group(nc)
    equip = overrides.get(nc, equipamento_from_patterns(nc))
    zona = tau_zona_from(equip, grp, nc)
    s = pd.to_numeric(df[col], errors="coerce")
    exemplo = s.dropna().iloc[0] if s.notna().any() else np.nan
    rows.append({
        "variavel": col,
        "variavel_normalizada": nc,
        "grupo_fisico": grp,
        "equipamento_processo": equip,
        "tau_zona": zona,
        "exemplo_valor": exemplo,
        "nulos": int(s.isna().sum()),
    })

inv = pd.DataFrame(rows).sort_values(
    ["tau_zona","grupo_fisico","equipamento_processo","variavel_normalizada"]
).reset_index(drop=True)

# Resumo após nova classificação
print("Contagem por tau_zona:")
print(inv["tau_zona"].value_counts(dropna=False))
print("\nContagem por grupo_fisico:")
print(inv["grupo_fisico"].value_counts(dropna=False))
print("\nAmostra (20):")
inv.head(20)


Contagem por tau_zona:
tau_zona
backpass       11
densa           8
ciclo_agua      7
indefinido      7
aux_maq         5
ciclo_vapor     4
emissoes        1
Name: count, dtype: int64

Contagem por grupo_fisico:
grupo_fisico
others       17
gas_path      7
bed           7
water         5
fan           4
steam         1
emissions     1
materials     1
Name: count, dtype: int64

Amostra (20):


,variavel,variavel_normalizada,grupo_fisico,equipamento_processo,tau_zona,exemplo_valor,nulos
0,temp_of_hpfa_fan_a_outl_adegc,temp_of_hpfa_fan_a_outl_adegc,fan,HPFA Fan A – Saída,aux_maq,95.351,1
1,temp_of_hpfa_fan_b_outl_adegc,temp_of_hpfa_fan_b_outl_adegc,fan,HPFA Fan B – Saída,aux_maq,95.351,1
2,temp_of_hpfa_fan_c_outl_adegc,temp_of_hpfa_fan_c_outl_adegc,fan,HPFA Fan C – Saída,aux_maq,23.507,0
3,te_of_hpfa_fan_outl_head_pipe_adegc,te_of_hpfa_fan_outl_head_pipe_adegc,fan,HPFA – Tubo de cabeçote – Saída,aux_maq,94.024,0
4,hi_temperture_reheol_pres_mpa_mi,hi_temperture_reheol_pres_mpa_mi,others,Aquecimento/óleo,aux_maq,0.006,0
5,temp_of_gas_in_aph_outl_adegc,temp_of_gas_in_aph_outl_adegc,gas_path,Gases – APH – Saída,backpass,118.027,0
6,te_of_gas_in_ecmz_side_a_outl_adegc,te_of_gas_in_ecmz_side_a_outl_adegc,gas_path,Gases – Economizer lado A – Saída,backpass,296.104,1
7,te_of_gas_in_ecmz_side_b_outl_adegc,te_of_gas_in_ecmz_side_b_outl_adegc,gas_path,Gases – Economizer lado B – Saída,backpass,300.422,0
8,temp_of_gas_in_ltr_side_a_inl_adegc,temp_of_gas_in_ltr_side_a_inl_adegc,gas_path,Gases – LTR lado A – Entrada,backpass,624.980,0
9,temp_of_gas_in_ltr_side_a_outl_adegc,temp_of_gas_in_ltr_side_a_outl_adegc,gas_path,Gases – LTR lado A – Saída,backpass,447.526,1


In [32]:
# RECONSTRUIR E SALVAR O DF CLASSIFICADO (com tau_zona)
from pathlib import Path
import pandas as pd
import numpy as np
import re, unicodedata

# Caminhos
PATH_PHYS = Path(r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\data\curated\a1_physics_informed.csv")
OUT_DIR   = Path(r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\outputs")
ARQ_OUT   = OUT_DIR / "inventario_classificado.csv"

# Util
def norm(s: str) -> str:
    s = "".join(ch for ch in unicodedata.normalize("NFKD", s) if not unicodedata.combining(ch))
    s = re.sub(r"\s+", "_", s.strip().lower())
    s = s.replace("-", "_")
    s = re.sub(r"[^a-z0-9_]", "", s)
    return re.sub(r"_+", "_", s).strip("_")

def is_temperature(nc: str) -> bool:
    if "temp" in nc or "temperature" in nc: return True
    if nc.endswith("degc") or nc.endswith("adeg"): return True
    return bool(re.search(r"(?:_|^)(?:temp|temperature)_(?:.*_)?(?:degc|adeg)(?:_|$)", nc))

# Leitura
if not PATH_PHYS.exists():
    raise FileNotFoundError(f"Arquivo não encontrado: {PATH_PHYS}")
df = pd.read_csv(PATH_PHYS, low_memory=False)
cols_norm = {c: norm(c) for c in df.columns}

# Colunas de temperatura
temp_cols = [orig for orig, nc in cols_norm.items() if is_temperature(nc)]

# Overrides finos (equipamento/processo)
overrides = {
    "bed_temperature_average_adegc": "Fornalha – Leito (média)",
    "furnace_bed_temp_1_3_sel_adegc":  "Fornalha – Leito (segmento 1–3)",
    "furnace_bed_temp_4_6_sel_adegc":  "Fornalha – Leito (segmento 4–6)",
    "furnace_bed_temp_7_9_sel_adegc":  "Fornalha – Leito (segmento 7–9)",
    "furnace_bed_temp_10_12_sel_adegc":"Fornalha – Leito (segmento 10–12)",
    "furnace_bed_temp_13_15_sel_adegc":"Fornalha – Leito (segmento 13–15)",
    "furnace_bed_temp_16_18_sel_adegc":"Fornalha – Leito (segmento 16–18)",
    "t_bfbp_b_inlet_temperature_adegc":"Fornalha – Leito inferior (seção B, entrada)",
    "temp_1_of_furnace_a_inner_adegc": "Fornalha A – Interna (ponto 1)",
    "temp_3_of_furnace_a_inner_adegc": "Fornalha A – Interna (ponto 3)",
    "temp_1_of_furnace_b_inner_adegc": "Fornalha B – Interna (ponto 1)",
    "temp_3_of_furnace_b_inner_adegc": "Fornalha B – Interna (ponto 3)",
    "temp_of_furnace_a_outl_adegc":    "Fornalha A – Saída de gases",
    "temp_of_furnace_b_outl_adegc":    "Fornalha B – Saída de gases",
    "temperature_of_cyclone_a_inlet_adegc": "Ciclone A – Entrada",
    "temperature_of_cyclone_b_inlet_adegc": "Ciclone B – Entrada",
    "temperature_of_cyclone_c_inlet_adegc": "Ciclone C – Entrada",
    "temp_of_gas_in_aph_outl_adegc":        "Gases – APH – Saída",
    "te_of_hot_pri_air_in_aph_outl_adegc":  "Ar de combustão – APH – Ar primário quente – Saída",
    "temp_of_gas_in_ltr_side_a_inl_adegc":  "Gases – LTR lado A – Entrada",
    "temp_of_gas_in_ltr_side_a_outl_adegc": "Gases – LTR lado A – Saída",
    "temp_of_gas_in_ltr_side_b_inl_adegc":  "Gases – LTR lado B – Entrada",
    "temp_of_gas_in_ltr_side_b_outl_adegc": "Gases – LTR lado B – Saída",
    "te_of_gas_in_ecmz_side_a_outl_adegc":  "Gases – Economizer lado A – Saída",
    "te_of_gas_in_ecmz_side_b_outl_adegc":  "Gases – Economizer lado B – Saída",
    "te_of_hpfa_fan_outl_head_pipe_adegc":  "HPFA – Tubo de cabeçote – Saída",
    "temp_of_hpfa_fan_a_outl_adegc":        "HPFA Fan A – Saída",
    "temp_of_hpfa_fan_b_outl_adegc":        "HPFA Fan B – Saída",
    "temp_of_hpfa_fan_c_outl_adegc":        "HPFA Fan C – Saída",
    "miain_steam_temperature_adegc":        "Vapor principal – Temperatura (verificar: main_steam)",
    "main_steam_temperature_adegc":         "Vapor principal – Temperatura",
    "hot_reheat_steam_temperature_1_adegc": "Vapor – Reheat quente (ponto 1)",
    "fsh_outl_hdr_temp_adegc_min":          "Vapor – Superaquecedor final – Cabeçote de saída (mín)",
    "hi_te_rh_ol_temp_adegc_min":           "Vapor – Reaquecedor alta temperatura – Saída (mín)",
    "temp_of_cnd_pump_outl_header_adegc":   "Ciclo água – Condensado – Bomba (coletor de saída)",
    "temp_of_cnd_water_of_dea_inl_adegc":   "Ciclo água – Condensado – Entrada do DEA",
    "temp_of_3_lph_outl_wtr_adegc":         "Ciclo água – LPH #3 – Saída de água",
    "temp_of_4_lph_outl_wtr_adegc":         "Ciclo água – LPH #4 – Saída de água",
    "te_no_6_hp_heater_ol_feed_wtr_adegc":  "Ciclo água – HP Heater #6 – Saída água de alimentação",
    "te_no_7_hp_heater_ol_feed_wtr_adegc":  "Ciclo água – HP Heater #7 – Saída água de alimentação",
    "te_no_8_hp_heater_ol_feed_wtr_adegc":  "Ciclo água – HP Heater #8 – Saída água de alimentação",
    "lms_pdr_bin_flud_air_htr_ol_te_adegc": "Calcário – Aquecedor ar de fluidização do silo – Saída",
    "esp_comb_with_ff_b_outl_temp_adegc":   "Emissões – ESP/FF lado B – Saída",
}

# Regras de macro-grupo
GROUP_RULES = [
    ("bed",      [r"(?:^|_)bed(?:_|$)", r"furnace_bed", r"\bfreeboard\b", r"\bbfb\b"]),
    ("cyclone",  [r"\bcyclone\b"]),
    ("fan",      [r"\bhpfa\b", r"\bsa_fan\b", r"(?:^|_)fan(?:_|$)"]),
    ("gas_path", [r"\baph\b", r"\bltr\b", r"\becmz\b", r"econom", r"(?:^|_)gas(?:_|$)"]),
    ("steam",    [r"main_steam", r"reheat", r"\bfsh\b", r"\bhot_reheat\b"]),
    ("water",    [r"\bdea\b", r"\bcnd\b", r"condens", r"\blph\b", r"feed_wtr", r"(?:^|_)wtr(?:_|$)"]),
    ("emissions",[r"(?:^|_)esp(?:_|$)", r"(?:^|_)ff(?:_|$)"]),
    ("materials",[r"\blms\b", r"limestone", r"flud_air_htr"]),
    ("furnace",  [r"\bfurnace\b"]),
]

def assign_group(nc: str) -> str:
    for group, pats in GROUP_RULES:
        for pat in pats:
            if re.search(pat, nc):
                return group
    return "others"

def equipamento_from_patterns(nc: str) -> str:
    if re.search(r"furnace_bed|(^|_)bed($|_)", nc): return "Fornalha – Leito"
    if re.search(r"furnace_.*_inner", nc):          return "Fornalha – Interna"
    if re.search(r"furnace_.*_outl", nc):           return "Fornalha – Saída de gases"
    if "cyclone" in nc and "inlet" in nc:           return "Ciclone – Entrada"
    if "aph" in nc and "gas" in nc:                 return "Gases – APH"
    if "ltr" in nc and "gas" in nc:                 return "Gases – LTR"
    if "ecmz" in nc or "econom" in nc:              return "Gases – Economizer"
    if "hpfa" in nc and "fan" in nc:                return "HPFA – Fan/Tubo"
    if "reheat" in nc or "rh" in nc:                return "Vapor – Reheat"
    if "main_steam" in nc or "miain_steam" in nc:   return "Vapor – Principal"
    if any(x in nc for x in ["dea","cnd","condens","lph","feed_wtr","wtr"]): return "Ciclo água"
    if any(x in nc for x in ["esp","ff"]):          return "Emissões – ESP/FF"
    if any(x in nc for x in ["oil","ol_"]):         return "Aquecimento/óleo"
    if "gas" in nc:                                 return "Caminho de gases"
    return "Outros"

def tau_zona_from(equip: str, grp: str, nc: str) -> str:
    if "Leito" in equip or grp == "bed":         return "densa"
    if ("Fornalha – Interna" in equip) or ("Fornalha – Saída" in equip): return "diluida"
    if grp in ["gas_path","cyclone"] or any(k in equip for k in ["APH","LTR","Economizer","Ciclone"]): return "backpass"
    if grp == "fan" or "HPFA" in equip or "óleo" in equip or "Aquec" in equip: return "aux_maq"
    if grp == "steam" or "Vapor" in equip:       return "ciclo_vapor"
    if grp == "water" or "Ciclo água" in equip:  return "ciclo_agua"
    if grp == "emissions" or "Emissões" in equip:return "emissoes"
    return "indefinido"

# Construir DF classificado
rows = []
for col in temp_cols:
    nc = cols_norm[col]
    grp = assign_group(nc)
    equip = overrides.get(nc, equipamento_from_patterns(nc))
    zona = tau_zona_from(equip, grp, nc)
    s = pd.to_numeric(df[col], errors="coerce")
    exemplo = s.dropna().iloc[0] if s.notna().any() else np.nan
    rows.append({
        "variavel": col,
        "variavel_normalizada": nc,
        "grupo_fisico": grp,
        "equipamento_processo": equip,
        "tau_zona": zona,
        "exemplo_valor": exemplo,
        "nulos": int(s.isna().sum()),
    })

inv = pd.DataFrame(rows).sort_values(
    ["tau_zona","grupo_fisico","equipamento_processo","variavel_normalizada"]
).reset_index(drop=True)

# Contagens (para confirmar sua saída)
print("Contagem por tau_zona:")
print(inv["tau_zona"].value_counts(dropna=False))
print("\nContagem por grupo_fisico:")
print(inv["grupo_fisico"].value_counts(dropna=False))

# Salvar
OUT_DIR.mkdir(parents=True, exist_ok=True)
inv.to_csv(ARQ_OUT, index=False, encoding="utf-8-sig")
print(f"\n✅ Inventário classificado salvo em: {ARQ_OUT}")


Contagem por tau_zona:
tau_zona
backpass       11
densa           8
ciclo_agua      7
aux_maq         6
indefinido      6
ciclo_vapor     4
emissoes        1
Name: count, dtype: int64

Contagem por grupo_fisico:
grupo_fisico
others       17
gas_path      7
bed           7
water         5
fan           4
materials     1
steam         1
emissions     1
Name: count, dtype: int64

✅ Inventário classificado salvo em: C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\outputs\inventario_classificado.csv


In [34]:
from pathlib import Path
import pandas as pd

BASE = Path(r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL")
ARQ_INV  = BASE / r"outputs\inventario_classificado.csv"
ARQ_PEND = BASE / r"outputs\pendencias_classificacao.csv"

if not ARQ_INV.exists():
    raise FileNotFoundError(f"Não encontrei {ARQ_INV}. Rode a célula que salva o inventário classificado.")

inv = pd.read_csv(ARQ_INV)

pend = inv[(inv["tau_zona"] == "indefinido") | (inv["grupo_fisico"] == "others")].copy()

ordem = ["grupo_fisico","tau_zona","equipamento_processo","variavel_normalizada","variavel"]
pend = pend[ [c for c in ordem if c in pend.columns] + 
             [c for c in ["exemplo_valor","nulos"] if c in pend.columns] ]

pend = pend.sort_values(by=[c for c in ordem if c in pend.columns])

ARQ_PEND.parent.mkdir(parents=True, exist_ok=True)
pend.to_csv(ARQ_PEND, index=False, encoding="utf-8-sig")

print(f"✅ Pendências salvas em: {ARQ_PEND}")
print(f"Total de pendências: {len(pend)}")
display(pend.head(20))


✅ Pendências salvas em: C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\outputs\pendencias_classificacao.csv
Total de pendências: 17


,grupo_fisico,tau_zona,equipamento_processo,variavel_normalizada,variavel,exemplo_valor,nulos
5,others,aux_maq,Aquecimento/óleo,hi_temperture_reheol_pres_mpa_mi,hi_temperture_reheol_pres_mpa_mi,0.006,0
13,others,backpass,Ar de combustão – APH – Ar primário quente – S...,te_of_hot_pri_air_in_aph_outl_adegc,te_of_hot_pri_air_in_aph_outl_adegc,261.133,0
14,others,backpass,Ciclone A – Entrada,temperature_of_cyclone_a_inlet_adegc,temperature_of_cyclone_a_inlet_adegc,745.441,2
15,others,backpass,Ciclone B – Entrada,temperature_of_cyclone_b_inlet_adegc,temperature_of_cyclone_b_inlet_adegc,729.978,1
16,others,backpass,Ciclone C – Entrada,temperature_of_cyclone_c_inlet_adegc,temperature_of_cyclone_c_inlet_adegc,769.650,10
17,others,ciclo_agua,Ciclo água – Condensado – Bomba (coletor de sa...,temp_of_cnd_pump_outl_header_adegc,temp_of_cnd_pump_outl_header_adegc,45.459,0
18,others,ciclo_agua,Ciclo água – Condensado – Entrada do DEA,temp_of_cnd_water_of_dea_inl_adegc,temp_of_cnd_water_of_dea_inl_adegc,69.035,0
24,others,ciclo_vapor,Vapor principal – Temperatura (verificar: main...,miain_steam_temperature_adegc,miain_steam_temperature_adegc,534.910,0
25,others,ciclo_vapor,Vapor – Reaquecedor alta temperatura – Saída (...,hi_te_rh_ol_temp_adegc_min,hi_te_rh_ol_temp_adegc_min,-0.358,0
26,others,ciclo_vapor,Vapor – Superaquecedor final – Cabeçote de saí...,fsh_outl_hdr_temp_adegc_min,fsh_outl_hdr_temp_adegc_min,-0.358,0



Corrigir a classificação com base no que já está na sua tabela (sem reler o CSV de dados).

Remover o intruso de pressão (hi_temperture_reheol_pres_mpa_mi → não é temperatura).

Definir grupo/tau onde ficou others/indefinido usando equipamento_processo.

Salvar de volta o inventário corrigido e gerar as pendências atualizadas.

In [35]:
# PATCH no inventário já salvo: corrige grupo/tau e remove intrusos
from pathlib import Path
import pandas as pd
import numpy as np

BASE = Path(r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL")
ARQ_INV  = BASE / r"outputs\inventario_classificado.csv"
ARQ_PEND = BASE / r"outputs\pendencias_classificacao.csv"

inv = pd.read_csv(ARQ_INV)

# 0) Remover intrusos que não são temperatura (ex.: pressão em MPa)
mask_nao_temp = inv["variavel_normalizada"].str.contains(r"(_|^)pres(_|$)|(_|^)mpa(_|$)", case=False, regex=True)
if mask_nao_temp.any():
    print("Removendo não-temperaturas (ex.: *_pres_* / *_mpa_*):", inv.loc[mask_nao_temp, "variavel"].tolist())
    inv = inv.loc[~mask_nao_temp].copy()

# Funções auxiliares
def set_if(mask, col, val):
    inv.loc[mask, col] = val

# 1) Corrigir grupo/tau a partir de 'equipamento_processo'
equip = inv["equipamento_processo"].fillna("")

# Ciclones → backpass / cyclone
m = equip.str.contains("Ciclone", case=False, regex=False)
set_if(m, "grupo_fisico", "cyclone")
set_if(m, "tau_zona",     "backpass")

# APH / LTR / Economizer / Caminho de gases → backpass / gas_path
m = equip.str.contains("APH|LTR|Economizer|Caminho de gases|Gases –", case=False, regex=True)
set_if(m, "grupo_fisico", "gas_path")
set_if(m, "tau_zona",     "backpass")

# Ciclo água → water / ciclo_agua
m = equip.str.contains("Ciclo água", case=False, regex=False)
set_if(m, "grupo_fisico", "water")
set_if(m, "tau_zona",     "ciclo_agua")

# Vapor → steam / ciclo_vapor
m = equip.str.contains("Vapor", case=False, regex=False)
set_if(m, "grupo_fisico", "steam")
set_if(m, "tau_zona",     "ciclo_vapor")

# Leito (bed) → bed / densa
m = equip.str.contains("Leito", case=False, regex=False)
set_if(m, "grupo_fisico", "bed")
set_if(m, "tau_zona",     "densa")

# Fornalha interna/saída → furnace / diluida
m = equip.str.contains("Fornalha .* Interna|Fornalha .* Saída", case=False, regex=True)
set_if(m, "grupo_fisico", "furnace")
set_if(m, "tau_zona",     "diluida")

# HPFA / óleo / aquecimento → fan (ou aux máquinas) / aux_maq
m = equip.str.contains("HPFA|Fan|Aquec", case=False, regex=True)
set_if(m, "grupo_fisico", "fan")
set_if(m, "tau_zona",     "aux_maq")

# Emissões (ESP/FF) → emissions / emissoes
m = equip.str.contains("Emissões|ESP/FF", case=False, regex=True)
set_if(m, "grupo_fisico", "emissions")
set_if(m, "tau_zona",     "emissoes")

# 2) Onde ainda ficou indefinido/others, tente resolver por variavel_normalizada (fallback leve)
vn = inv["variavel_normalizada"].fillna("")

# Furnace bed/by name
m = vn.str.contains(r"(?:^|_)bed(?:_|$)|furnace_bed", case=False, regex=True)
set_if(m, "grupo_fisico", "bed")
set_if(m, "tau_zona",     "densa")

# Furnace inner/outl
m = vn.str.contains(r"furnace_.*_inner|furnace_.*_outl", case=False, regex=True)
set_if(m, "grupo_fisico", "furnace")
set_if(m, "tau_zona",     "diluida")

# Cyclone inlet
m = vn.str.contains(r"cyclone_.*_inlet", case=False, regex=True)
set_if(m, "grupo_fisico", "cyclone")
set_if(m, "tau_zona",     "backpass")

# LTR/APH/ECMZ/econom
m = vn.str.contains(r"(?:^|_)(aph|ltr|ecmz|econom)", case=False, regex=True)
set_if(m, "grupo_fisico", "gas_path")
set_if(m, "tau_zona",     "backpass")

# main_steam/reheat/fsh
m = vn.str.contains(r"(main_steam|reheat|^fsh_|hot_reheat)", case=False, regex=True)
set_if(m, "grupo_fisico", "steam")
set_if(m, "tau_zona",     "ciclo_vapor")

# dea/cnd/lph/wtr
m = vn.str.contains(r"(?:^|_)(dea|cnd|condens|lph|feed_wtr|wtr)(?:_|$)", case=False, regex=True)
set_if(m, "grupo_fisico", "water")
set_if(m, "tau_zona",     "ciclo_agua")

# hpfa/fan
m = vn.str.contains(r"(?:^|_)(hpfa|fan)(?:_|$)", case=False, regex=True)
set_if(m, "grupo_fisico", "fan")
set_if(m, "tau_zona",     "aux_maq")

# esp|ff
m = vn.str.contains(r"(?:^|_)(esp|ff)(?:_|$)", case=False, regex=True)
set_if(m, "grupo_fisico", "emissions")
set_if(m, "tau_zona",     "emissoes")

# 3) Relatórios
print("Contagem por tau_zona (após patch):")
print(inv["tau_zona"].value_counts(dropna=False))
print("\nContagem por grupo_fisico (após patch):")
print(inv["grupo_fisico"].value_counts(dropna=False))

# 4) Salvar inventário corrigido e pendências
inv.to_csv(ARQ_INV, index=False, encoding="utf-8-sig")

pend = inv[(inv["tau_zona"] == "indefinido") | (inv["grupo_fisico"] == "others")].copy()
pend = pend[["grupo_fisico","tau_zona","equipamento_processo","variavel_normalizada","variavel","exemplo_valor","nulos"]]
pend = pend.sort_values(["grupo_fisico","tau_zona","variavel_normalizada"])

ARQ_PEND.parent.mkdir(parents=True, exist_ok=True)
pend.to_csv(ARQ_PEND, index=False, encoding="utf-8-sig")

print(f"\n✅ Inventário corrigido salvo em: {ARQ_INV}")
print(f"📝 Pendências atualizadas em: {ARQ_PEND} (total: {len(pend)})")
display(pend.head(20))


Removendo não-temperaturas (ex.: *_pres_* / *_mpa_*): ['hi_temperture_reheol_pres_mpa_mi']
Contagem por tau_zona (após patch):
tau_zona
backpass       11
densa           8
ciclo_agua      7
aux_maq         6
diluida         6
ciclo_vapor     3
emissoes        1
Name: count, dtype: int64

Contagem por grupo_fisico (após patch):
grupo_fisico
gas_path     8
bed          8
water        7
fan          6
furnace      6
cyclone      3
steam        3
emissions    1
Name: count, dtype: int64

✅ Inventário corrigido salvo em: C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\outputs\inventario_classificado.csv
📝 Pendências atualizadas em: C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\outputs\pendencias_classificacao.csv (total: 0)


C:\Users\wilso\AppData\Local\Temp\ipykernel_9748\3674370342.py:13: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask_nao_temp = inv["variavel_normalizada"].str.contains(r"(_|^)pres(_|$)|(_|^)mpa(_|$)", case=False, regex=True)
C:\Users\wilso\AppData\Local\Temp\ipykernel_9748\3674370342.py:84: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  m = vn.str.contains(r"(?:^|_)(aph|ltr|ecmz|econom)", case=False, regex=True)
C:\Users\wilso\AppData\Local\Temp\ipykernel_9748\3674370342.py:89: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  m = vn.str.contains(r"(main_steam|reheat|^fsh_|hot_reheat)", case=False, regex=True)
C:\Users\wilso\AppData\Local\Temp\ipykernel_9748\3674370342.py:94: UserWarning: This pattern is interpreted as a regul

,grupo_fisico,tau_zona,equipamento_processo,variavel_normalizada,variavel,exemplo_valor,nulos


Otimo — agora que o inventário ficou zerado de pendências e salvo, vamos fechar a etapa dos tau_* e deixar a base pronta pro physics_based.

Abaixo vai um pacote direto ao ponto:

cria o mapeamento zona → colunas a partir do inventario_classificado.csv

calcula tau_densa, tau_diluida, tau_backpass (média de z‑scores por zona)

opcional: tau_global = média dos três tau_*

salva:

outputs/tau_mapping.csv (auditoria)

data/curated/a1_physics_informed_enriched.csv (dataset pronto)

Observação: usei z‑score por coluna (ignorando NaN, tratando coluna constante como NaN) e média por linha. É determinístico e auditável.

In [36]:
# MAPEAMENTO TAU (a partir do inventário classificado)
from pathlib import Path
import pandas as pd

BASE = Path(r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL")
ARQ_INV   = BASE / r"outputs\inventario_classificado.csv"
ARQ_MAP   = BASE / r"outputs\tau_mapping.csv"

inv = pd.read_csv(ARQ_INV)

# zonas que compõem tau_*
zonas_para_tau = {
    "densa": "tau_densa",
    "diluida": "tau_diluida",
    "backpass": "tau_backpass",
}

# construir mapping zona -> lista de colunas (nomes originais do dataset)
zona_cols = {
    z: inv.loc[inv["tau_zona"] == z, "variavel"].dropna().tolist()
    for z in zonas_para_tau.keys()
}

# salvar auditoria do mapping
rows = []
for z, cols in zona_cols.items():
    for c in cols:
        rows.append({"tau_zona": z, "coluna": c})
tau_map_df = pd.DataFrame(rows).sort_values(["tau_zona","coluna"])
ARQ_MAP.parent.mkdir(parents=True, exist_ok=True)
tau_map_df.to_csv(ARQ_MAP, index=False, encoding="utf-8-sig")

print("✅ Mapping salvo em:", ARQ_MAP)
for z, cols in zona_cols.items():
    print(f"- {z}: {len(cols)} colunas")


✅ Mapping salvo em: C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\outputs\tau_mapping.csv
- densa: 8 colunas
- diluida: 6 colunas
- backpass: 11 colunas


In [37]:
# CÁLCULO DOS TAU_* E SALVAMENTO DA BASE ENRIQUECIDA
import numpy as np

ARQ_BASE_IN  = BASE / r"data\curated\a1_physics_informed.csv"
ARQ_BASE_OUT = BASE / r"data\curated\a1_physics_informed_enriched.csv"

df = pd.read_csv(ARQ_BASE_IN, low_memory=False)

def zscore(series: pd.Series) -> pd.Series:
    x = pd.to_numeric(series, errors="coerce")
    mu = x.mean()
    sd = x.std(ddof=1)
    if not np.isfinite(sd) or sd == 0:
        # coluna constante ou inválida -> retorna NaN
        return pd.Series(np.nan, index=x.index)
    return (x - mu) / sd

# calcular cada tau_* a partir do mapping
for z, outcol in zonas_para_tau.items():
    cols = [c for c in zona_cols[z] if c in df.columns]
    if not cols:
        df[outcol] = np.nan
        print(f"{outcol}: nenhuma coluna elegível.")
        continue
    Z = pd.DataFrame({c: zscore(df[c]) for c in cols})
    df[outcol] = Z.mean(axis=1, skipna=True)
    print(f"{outcol}: calculado a partir de {len(cols)} colunas.")

# (opcional) tau_global = média dos três tau_*
df["tau_global"] = df[list(zonas_para_tau.values())].mean(axis=1, skipna=True)

# salvar
df.to_csv(ARQ_BASE_OUT, index=False, encoding="utf-8-sig")
print("\n✅ Base enriquecida salva em:", ARQ_BASE_OUT)

# pequena prévia
preview_cols = ["Timestamp"] if "Timestamp" in df.columns else []
preview_cols += list(zonas_para_tau.values()) + ["tau_global"]
print("\nPrévia (top 5):")
print(df[preview_cols].head(5))


tau_densa: calculado a partir de 8 colunas.
tau_diluida: calculado a partir de 6 colunas.
tau_backpass: calculado a partir de 11 colunas.

✅ Base enriquecida salva em: C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\data\curated\a1_physics_informed_enriched.csv

Prévia (top 5):
             Timestamp  tau_densa  tau_diluida  tau_backpass  tau_global
0  2023-03-01 07:00:00  -1.967493    -1.146757     -1.074136   -1.396129
1  2023-03-01 08:00:00  -1.795885    -1.047888     -0.831629   -1.225134
2  2023-03-01 09:00:00  -0.904943    -0.355473     -0.062435   -0.440951
3  2023-03-01 10:00:00  -0.775310    -0.036760      0.509053   -0.101006
4  2023-03-01 11:00:00  -0.252939     0.232029      0.369616    0.116236


In [38]:
# RELATÓRIO RÁPIDO DA QUALIDADE DOS TAU_*
tau_cols = list(zonas_para_tau.values()) + ["tau_global"]
qual = []
for c in tau_cols:
    s = pd.to_numeric(df[c], errors="coerce")
    qual.append({
        "coluna": c,
        "n": int(s.notna().sum()),
        "nulos": int(s.isna().sum()),
        "min": float(np.nanmin(s)) if s.notna().any() else np.nan,
        "p50": float(np.nanmedian(s)) if s.notna().any() else np.nan,
        "max": float(np.nanmax(s)) if s.notna().any() else np.nan,
        "mean": float(np.nanmean(s)) if s.notna().any() else np.nan,
        "std": float(np.nanstd(s, ddof=1)) if s.notna().any() else np.nan,
    })
pd.DataFrame(qual)


,coluna,n,nulos,min,p50,max,mean,std
0,tau_densa,11757,0,-4.022279,-0.099158,1.693458,-0.014584,0.567571
1,tau_diluida,11757,0,-3.955045,0.125038,1.369591,-0.000004,0.605662
2,tau_backpass,11757,0,-4.083717,0.072231,1.427059,-0.000030,0.479271
3,tau_global,11757,0,-3.690811,0.095584,1.294544,-0.004873,0.416325


Introdução conceitual (baseline físico)

A ideia do baseline físico é fixar um “retrato de referência” do regime térmico do CFB em operação estável, contra o qual compararemos variações futuras.
Usaremos os proxies:

     tau_densa, tau_diluida, tau_backpass (e tau_global) — regime térmico por zonas.

     (quando existirem) v_proxy — intensidade/carga global.

     (quando existirem) delta_proxy — regime fluidodinâmico (ΔP).

Critério de estabilidade (auditatório e ajustável):

     filtrar outliers grossos por quantis (ex.: 1%–99%) dos tau_*;

     exigir baixa variabilidade local (desvio-padrão rolante pequeno) nas séries de tau_*;

     exigir persistência mínima do estado estável (ex.: janelas consecutivas de N amostras).

Com isso, geramos:

     physics_baseline.csv → médias e dispersões dos proxies no estado estável;

     baseline_janelas.csv → janelas/intervalos temporais que compuseram o baseline (auditoria).

In [40]:
# ETAPA 0 — PREPARO
from pathlib import Path
import pandas as pd
import numpy as np

BASE = Path(r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL")
ARQ_DADOS = BASE / r"data\curated\a1_physics_informed_enriched.csv"  # saída da etapa anterior
ARQ_BASELINE = BASE / r"outputs\physics_baseline.csv"
ARQ_JANELAS  = BASE / r"outputs\baseline_janelas.csv"

if not ARQ_DADOS.exists():
    raise FileNotFoundError(f"SEM EVIDÊNCIA OBJETIVA – não encontrei {ARQ_DADOS}. Gere antes o dataset enriquecido.")


In [41]:
# ETAPA 1 — LEITURA E CHECKS
df = pd.read_csv(ARQ_DADOS, low_memory=False)

# Timestamp
ts_col = "Timestamp" if "Timestamp" in df.columns else None
if ts_col:
    df[ts_col] = pd.to_datetime(df[ts_col], errors="coerce")
    df = df.sort_values(ts_col).reset_index(drop=True)

# Proxies térmicos obrigatórios para esta etapa
req_tau = ["tau_densa","tau_diluida","tau_backpass","tau_global"]
faltando_tau = [c for c in req_tau if c not in df.columns]
if faltando_tau:
    raise KeyError(f"Faltam colunas tau para baseline: {faltando_tau}")

# Proxies opcionais
opt_cols = [c for c in ["v_proxy","delta_proxy"] if c in df.columns]

print("Dimensões:", df.shape)
print("Proxies térmicos OK:", [c for c in req_tau if c in df.columns])
print("Proxies opcionais encontrados:", opt_cols)


Dimensões: (11757, 154)
Proxies térmicos OK: ['tau_densa', 'tau_diluida', 'tau_backpass', 'tau_global']
Proxies opcionais encontrados: []


In [42]:
# ETAPA 2 — LIMPEZA POR QUANTIS (ajustável)
taus = ["tau_densa","tau_diluida","tau_backpass","tau_global"]
q_low, q_high = 0.01, 0.99

mask_quantis = pd.Series(True, index=df.index)
for c in taus:
    s = pd.to_numeric(df[c], errors="coerce")
    lo, hi = s.quantile(q_low), s.quantile(q_high)
    mask_c = s.between(lo, hi, inclusive="both")
    mask_quantis &= mask_c

df_q = df.loc[mask_quantis].copy()
print(f"Removidos por quantis: {len(df) - len(df_q)} de {len(df)} linhas")


Removidos por quantis: 677 de 11757 linhas


In [43]:
# ETAPA 3 — ESTABILIDADE POR ROLLING STD
win = 12   # tamanho da janela (ex.: 12 amostras; ajuste para sua taxa de amostragem)
thr = 0.50 # limiar de std aceitável para cada tau_* (ajustável)

df_s = df_q.copy()
for c in taus:
    df_s[f"{c}_stdroll"] = pd.to_numeric(df_s[c], errors="coerce").rolling(win, min_periods=win).std()

# estável se todos os stdroll < thr
std_cols = [f"{c}_stdroll" for c in taus]
mask_stable = (df_s[std_cols] < thr).all(axis=1)

df_estavel = df_s.loc[mask_stable].copy()
print("Candidatos a estável:", len(df_estavel))


Candidatos a estável: 10654


In [46]:
# ETAPA 4 — PERSISTÊNCIA MÍNIMA
min_run = 168  # número mínimo de amostras consecutivas estáveis (ajuste conforme sua cadência)

stable_idx = df_s.index[mask_stable]
if stable_idx.empty:
    raise RuntimeError("Nenhum ponto estável encontrado com os parâmetros atuais. Ajuste 'win', 'thr' ou quantis.")

# identificar blocos consecutivos
blocks = []
start = prev = None
for i in stable_idx:
    if start is None:
        start = prev = i
    elif i == prev + 1:
        prev = i
    else:
        blocks.append((start, prev))
        start = prev = i
if start is not None:
    blocks.append((start, prev))

# filtrar por tamanho mínimo
blocks_ok = [(a,b) for (a,b) in blocks if (b - a + 1) >= min_run]

print(f"Blocos estáveis encontrados: {len(blocks)} | Válidos (min_run={min_run}): {len(blocks_ok)}")

# construir dataframe de janelas (auditoria)
jrows = []
for a,b in blocks_ok:
    sl = df_s.loc[a:b]
    if ts_col:
        jrows.append({"ini_idx":a,"fim_idx":b,"n":(b-a+1),
                      "ini_ts":sl[ts_col].iloc[0], "fim_ts":sl[ts_col].iloc[-1]})
    else:
        jrows.append({"ini_idx":a,"fim_idx":b,"n":(b-a+1)})
df_jan = pd.DataFrame(jrows).sort_values("ini_idx").reset_index(drop=True)
df_jan.head()


Blocos estáveis encontrados: 330 | Válidos (min_run=168): 15


,ini_idx,fim_idx,n,ini_ts,fim_ts
0,763,1462,700,2023-06-03 12:00:00,2023-07-03 19:00:00
1,1464,1831,368,2023-07-04 05:00:00,2023-07-19 12:00:00
2,2521,2701,181,2023-08-17 14:00:00,2023-08-25 02:00:00
3,3151,3368,218,2023-10-14 17:00:00,2023-10-23 18:00:00
4,3589,3822,234,2023-11-01 23:00:00,2023-11-11 18:00:00


In [47]:
# ETAPA 5 — CÁLCULO E SALVAMENTO DO BASELINE
sel_idx = []
for a,b in blocks_ok:
    sel_idx.extend(range(a, b+1))
sel_idx = sorted(set(sel_idx))

base = df_s.loc[sel_idx].copy()

metric_cols = taus + [c for c in ["v_proxy","delta_proxy"] if c in df_s.columns]

resumo = []
for c in metric_cols:
    s = pd.to_numeric(base[c], errors="coerce")
    resumo.append({
        "variavel": c,
        "n": int(s.notna().sum()),
        "media": float(np.nanmean(s)) if s.notna().any() else np.nan,
        "std": float(np.nanstd(s, ddof=1)) if s.notna().any() else np.nan,
        "q05": float(np.nanquantile(s, 0.05)) if s.notna().any() else np.nan,
        "q50": float(np.nanmedian(s)) if s.notna().any() else np.nan,
        "q95": float(np.nanquantile(s, 0.95)) if s.notna().any() else np.nan,
    })

baseline_df = pd.DataFrame(resumo)

# salvar
ARQ_BASELINE.parent.mkdir(parents=True, exist_ok=True)
baseline_df.to_csv(ARQ_BASELINE, index=False, encoding="utf-8-sig")
df_jan.to_csv(ARQ_JANELAS, index=False, encoding="utf-8-sig")

print(f"✅ Baseline salvo em: {ARQ_BASELINE}")
print(f"🧾 Janelas (auditoria) salvas em: {ARQ_JANELAS}")
baseline_df


✅ Baseline salvo em: C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\outputs\physics_baseline.csv
🧾 Janelas (auditoria) salvas em: C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\outputs\baseline_janelas.csv


,variavel,n,media,std,q05,q50,q95
0,tau_densa,4797,0.016389,0.511947,-0.686572,-0.106453,0.980413
1,tau_diluida,4797,-0.009938,0.521204,-0.822882,0.055395,0.667885
2,tau_backpass,4797,-0.007671,0.398794,-0.678915,0.079976,0.551073
3,tau_global,4797,-0.000407,0.333599,-0.600432,0.095966,0.426512


perfeito — 7 dias (168 amostras) ficou um critério sólido. Vamos fechar o baseline com duas referências complementares e tudo auditado:

Baseline_Longest: usa apenas o maior bloco estável (retrato “ouro”).

Baseline_Consensus: usa todos os blocos válidos (estatística robusta por mediana).

A seguir, textos + células para: consolidar janelas, calcular os dois baselines, e salvar tudo.



In [48]:
# ETAPA 5B — CONSOLIDAÇÃO POR BLOCO (resumo por janela estável)
import numpy as np

# pré-requisitos: df_s (com *_stdroll) e df_jan, taus, opt_cols, ts_col
metric_cols = ["tau_densa","tau_diluida","tau_backpass","tau_global"] + [c for c in ["v_proxy","delta_proxy"] if c in df_s.columns]

res_blocos = []
for _, r in df_jan.iterrows():
    a, b = int(r["ini_idx"]), int(r["fim_idx"])
    sl = df_s.loc[a:b, metric_cols]
    row = {
        "ini_idx": a,
        "fim_idx": b,
        "n": int(b - a + 1),
        "ini_ts": r.get("ini_ts"),
        "fim_ts": r.get("fim_ts"),
    }
    for c in metric_cols:
        s = pd.to_numeric(sl[c], errors="coerce")
        row[f"{c}__mean"] = float(np.nanmean(s))
        row[f"{c}__std"]  = float(np.nanstd(s, ddof=1))
        row[f"{c}__p50"]  = float(np.nanmedian(s))
        row[f"{c}__q05"]  = float(np.nanquantile(s, 0.05))
        row[f"{c}__q95"]  = float(np.nanquantile(s, 0.95))
    res_blocos.append(row)

df_blocos = pd.DataFrame(res_blocos).sort_values("n", ascending=False).reset_index(drop=True)
print("Top 5 blocos por duração:")
df_blocos.head(5)


Top 5 blocos por duração:


,ini_idx,fim_idx,n,ini_ts,fim_ts,tau_densa__mean,tau_densa__std,tau_densa__p50,tau_densa__q05,tau_densa__q95,tau_diluida__mean,tau_diluida__std,tau_diluida__p50,tau_diluida__q05,tau_diluida__q95,tau_backpass__mean,tau_backpass__std,tau_backpass__p50,tau_backpass__q05,tau_backpass__q95,tau_global__mean,tau_global__std,tau_global__p50,tau_global__q05,tau_global__q95
0,763,1462,700,2023-06-03 12:00:00,2023-07-03 19:00:00,-0.209954,0.188616,-0.204079,-0.507573,0.099983,0.415141,0.127123,0.418898,0.190638,0.611054,0.311724,0.158179,0.299722,0.083815,0.603869,0.172304,0.131499,0.166804,-0.032080,0.397780
1,10905,11389,485,2024-11-20 23:00:00,2024-12-11 09:00:00,0.393661,0.299434,0.440936,-0.138502,0.815435,-0.255685,0.244310,-0.227889,-0.650841,0.120456,-0.129588,0.251981,-0.120674,-0.542528,0.269980,0.002796,0.240737,0.049626,-0.412588,0.364007
2,7000,7408,409,2024-05-06 14:00:00,2024-05-23 14:00:00,0.910072,0.196400,0.936660,0.665909,1.137378,-0.088118,0.232850,-0.080460,-0.449007,0.272374,0.094568,0.146474,0.094164,-0.136031,0.309636,0.305507,0.159150,0.325583,0.110735,0.500158
3,5388,5775,388,2024-01-31 22:00:00,2024-02-17 01:00:00,-0.296202,0.258788,-0.263347,-0.708787,0.072945,-0.640175,0.158341,-0.628161,-0.908583,-0.401760,-0.551208,0.174286,-0.547587,-0.813383,-0.274374,-0.495862,0.154339,-0.487303,-0.745460,-0.241863
4,1464,1831,368,2023-07-04 05:00:00,2023-07-19 12:00:00,-0.211092,0.226439,-0.192618,-0.686454,0.122923,0.510496,0.123204,0.507877,0.335305,0.730113,0.436680,0.158169,0.432679,0.209106,0.716580,0.245361,0.139317,0.252415,0.011186,0.458271


Consolida as janelas estáveis válidas (min_run=168), cria uma tabela por bloco com métricas de tau_* (+ proxies opcionais) e ordena por duração.

In [49]:
# ETAPA 5B — CONSOLIDAÇÃO POR BLOCO (resumo por janela estável)
import numpy as np

# pré-requisitos: df_s (com *_stdroll) e df_jan, taus, opt_cols, ts_col
metric_cols = ["tau_densa","tau_diluida","tau_backpass","tau_global"] + [c for c in ["v_proxy","delta_proxy"] if c in df_s.columns]

res_blocos = []
for _, r in df_jan.iterrows():
    a, b = int(r["ini_idx"]), int(r["fim_idx"])
    sl = df_s.loc[a:b, metric_cols]
    row = {
        "ini_idx": a,
        "fim_idx": b,
        "n": int(b - a + 1),
        "ini_ts": r.get("ini_ts"),
        "fim_ts": r.get("fim_ts"),
    }
    for c in metric_cols:
        s = pd.to_numeric(sl[c], errors="coerce")
        row[f"{c}__mean"] = float(np.nanmean(s))
        row[f"{c}__std"]  = float(np.nanstd(s, ddof=1))
        row[f"{c}__p50"]  = float(np.nanmedian(s))
        row[f"{c}__q05"]  = float(np.nanquantile(s, 0.05))
        row[f"{c}__q95"]  = float(np.nanquantile(s, 0.95))
    res_blocos.append(row)

df_blocos = pd.DataFrame(res_blocos).sort_values("n", ascending=False).reset_index(drop=True)
print("Top 5 blocos por duração:")
df_blocos.head(5)


Top 5 blocos por duração:


,ini_idx,fim_idx,n,ini_ts,fim_ts,tau_densa__mean,tau_densa__std,tau_densa__p50,tau_densa__q05,tau_densa__q95,tau_diluida__mean,tau_diluida__std,tau_diluida__p50,tau_diluida__q05,tau_diluida__q95,tau_backpass__mean,tau_backpass__std,tau_backpass__p50,tau_backpass__q05,tau_backpass__q95,tau_global__mean,tau_global__std,tau_global__p50,tau_global__q05,tau_global__q95
0,763,1462,700,2023-06-03 12:00:00,2023-07-03 19:00:00,-0.209954,0.188616,-0.204079,-0.507573,0.099983,0.415141,0.127123,0.418898,0.190638,0.611054,0.311724,0.158179,0.299722,0.083815,0.603869,0.172304,0.131499,0.166804,-0.032080,0.397780
1,10905,11389,485,2024-11-20 23:00:00,2024-12-11 09:00:00,0.393661,0.299434,0.440936,-0.138502,0.815435,-0.255685,0.244310,-0.227889,-0.650841,0.120456,-0.129588,0.251981,-0.120674,-0.542528,0.269980,0.002796,0.240737,0.049626,-0.412588,0.364007
2,7000,7408,409,2024-05-06 14:00:00,2024-05-23 14:00:00,0.910072,0.196400,0.936660,0.665909,1.137378,-0.088118,0.232850,-0.080460,-0.449007,0.272374,0.094568,0.146474,0.094164,-0.136031,0.309636,0.305507,0.159150,0.325583,0.110735,0.500158
3,5388,5775,388,2024-01-31 22:00:00,2024-02-17 01:00:00,-0.296202,0.258788,-0.263347,-0.708787,0.072945,-0.640175,0.158341,-0.628161,-0.908583,-0.401760,-0.551208,0.174286,-0.547587,-0.813383,-0.274374,-0.495862,0.154339,-0.487303,-0.745460,-0.241863
4,1464,1831,368,2023-07-04 05:00:00,2023-07-19 12:00:00,-0.211092,0.226439,-0.192618,-0.686454,0.122923,0.510496,0.123204,0.507877,0.335305,0.730113,0.436680,0.158169,0.432679,0.209106,0.716580,0.245361,0.139317,0.252415,0.011186,0.458271


In [50]:
# ETAPA 5B — CONSOLIDAÇÃO POR BLOCO (resumo por janela estável)
import numpy as np

# pré-requisitos: df_s (com *_stdroll) e df_jan, taus, opt_cols, ts_col
metric_cols = ["tau_densa","tau_diluida","tau_backpass","tau_global"] + [c for c in ["v_proxy","delta_proxy"] if c in df_s.columns]

res_blocos = []
for _, r in df_jan.iterrows():
    a, b = int(r["ini_idx"]), int(r["fim_idx"])
    sl = df_s.loc[a:b, metric_cols]
    row = {
        "ini_idx": a,
        "fim_idx": b,
        "n": int(b - a + 1),
        "ini_ts": r.get("ini_ts"),
        "fim_ts": r.get("fim_ts"),
    }
    for c in metric_cols:
        s = pd.to_numeric(sl[c], errors="coerce")
        row[f"{c}__mean"] = float(np.nanmean(s))
        row[f"{c}__std"]  = float(np.nanstd(s, ddof=1))
        row[f"{c}__p50"]  = float(np.nanmedian(s))
        row[f"{c}__q05"]  = float(np.nanquantile(s, 0.05))
        row[f"{c}__q95"]  = float(np.nanquantile(s, 0.95))
    res_blocos.append(row)

df_blocos = pd.DataFrame(res_blocos).sort_values("n", ascending=False).reset_index(drop=True)
print("Top 5 blocos por duração:")
df_blocos.head(5)


Top 5 blocos por duração:


,ini_idx,fim_idx,n,ini_ts,fim_ts,tau_densa__mean,tau_densa__std,tau_densa__p50,tau_densa__q05,tau_densa__q95,tau_diluida__mean,tau_diluida__std,tau_diluida__p50,tau_diluida__q05,tau_diluida__q95,tau_backpass__mean,tau_backpass__std,tau_backpass__p50,tau_backpass__q05,tau_backpass__q95,tau_global__mean,tau_global__std,tau_global__p50,tau_global__q05,tau_global__q95
0,763,1462,700,2023-06-03 12:00:00,2023-07-03 19:00:00,-0.209954,0.188616,-0.204079,-0.507573,0.099983,0.415141,0.127123,0.418898,0.190638,0.611054,0.311724,0.158179,0.299722,0.083815,0.603869,0.172304,0.131499,0.166804,-0.032080,0.397780
1,10905,11389,485,2024-11-20 23:00:00,2024-12-11 09:00:00,0.393661,0.299434,0.440936,-0.138502,0.815435,-0.255685,0.244310,-0.227889,-0.650841,0.120456,-0.129588,0.251981,-0.120674,-0.542528,0.269980,0.002796,0.240737,0.049626,-0.412588,0.364007
2,7000,7408,409,2024-05-06 14:00:00,2024-05-23 14:00:00,0.910072,0.196400,0.936660,0.665909,1.137378,-0.088118,0.232850,-0.080460,-0.449007,0.272374,0.094568,0.146474,0.094164,-0.136031,0.309636,0.305507,0.159150,0.325583,0.110735,0.500158
3,5388,5775,388,2024-01-31 22:00:00,2024-02-17 01:00:00,-0.296202,0.258788,-0.263347,-0.708787,0.072945,-0.640175,0.158341,-0.628161,-0.908583,-0.401760,-0.551208,0.174286,-0.547587,-0.813383,-0.274374,-0.495862,0.154339,-0.487303,-0.745460,-0.241863
4,1464,1831,368,2023-07-04 05:00:00,2023-07-19 12:00:00,-0.211092,0.226439,-0.192618,-0.686454,0.122923,0.510496,0.123204,0.507877,0.335305,0.730113,0.436680,0.158169,0.432679,0.209106,0.716580,0.245361,0.139317,0.252415,0.011186,0.458271


perfeito — vamos fechar o Baseline Físico Interseccional com os três pilares, do jeito que você determinou:

τ estável (τ_densa, τ_diluida, τ_backpass, τ_global)

carga de carvão = Flow C → flw_total_c_t_h (proxy oficial de carga no forno)

ar total = Ar primário + Ar secundário (Nm³/h; aceitar Nm³/min e converter; ignorar totalmente temperaturas de fans)

opcional: se existirem ΔP/pressões de fans relevantes, podemos exigir estabilidade adicional nelas (sem usar temperatura de saída).

Abaixo vai o texto + células. Tudo sobrescreve apenas saídas em outputs/ e não altera seus dados originais.

Conceito (curto)
O baseline físico deve refletir estabilidade térmica e, ao mesmo tempo, ponto de operação representativo: combustível sendo de fato queimado no forno (Flow C adequado) e combustão/ fluidização com ar correto (ar total em faixa). Assim, evitamos congelar como referência períodos de meia-carga, excesso de ar, ou qualquer regime “estável porém não representativo”.

0) Imports, caminhos e utilitários
O que faz: prepara ambiente, normalização de nomes e helpers para detectar colunas de ar (primário/ secundário) por unidade e semântica.

In [51]:
# ETAPA 0 — PREPARO
from pathlib import Path
import pandas as pd
import numpy as np
import re, unicodedata

BASE = Path(r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL")
ARQ_DADOS = BASE / r"data\curated\a1_physics_informed_enriched.csv"

# saídas (interseção)
ARQ_BASELINE_INT = BASE / r"outputs\physics_baseline_intersection.csv"
ARQ_JANELAS_INT  = BASE / r"outputs\baseline_janelas_intersection.csv"
ARQ_README_INT   = BASE / r"outputs\baseline_intersection_readme.md"

if not ARQ_DADOS.exists():
    raise FileNotFoundError(f"Não encontrei {ARQ_DADOS}. Gere antes a base enriquecida com tau_*.")

def norm(s: str) -> str:
    s = "".join(ch for ch in unicodedata.normalize("NFKD", s) if not unicodedata.combining(ch))
    s = re.sub(r"\s+", "_", s.strip().lower())
    s = s.replace("-", "_")
    s = re.sub(r"[^a-z0-9_]", "", s)
    return re.sub(r"_+", "_", s).strip("_")


1) Leitura, sanity e detecção de Flow C e Ar total
O que faz: lê a base; garante presença de tau_* e Flow C; identifica colunas de ar primário/ secundário em Nm³/h ou Nm³/min, soma e converte tudo para Nm³/h.

In [52]:
# ETAPA 1 — LEITURA E DETECÇÃO DE PROXIES
df = pd.read_csv(ARQ_DADOS, low_memory=False)
ts_col = "Timestamp" if "Timestamp" in df.columns else None
if ts_col:
    df[ts_col] = pd.to_datetime(df[ts_col], errors="coerce")
    df = df.sort_values(ts_col).reset_index(drop=True)

# obrigatórios para τ
taus = ["tau_densa","tau_diluida","tau_backpass","tau_global"]
faltam = [c for c in taus if c not in df.columns]
if faltam:
    raise KeyError(f"Faltam colunas tau_* para o baseline: {faltam}")

# proxy de carvão: Flow C
FLOWC = "flw_total_c_t_h"
if FLOWC not in df.columns:
    raise KeyError(f"Proxy de carga (Flow C) não encontrado: {FLOWC}")

# detecção de ar primário/ secundário por nome e unidade
cols_norm = {c: norm(c) for c in df.columns}

def is_nm3_per_hour(nc: str) -> bool:
    return nc.endswith("nm3_h") or nc.endswith("nm3h")

def is_nm3_per_min(nc: str) -> bool:
    return nc.endswith("nm3_min") or nc.endswith("nm3min")

def is_primary_air(nc: str) -> bool:
    # termos comuns: primary, pri, pa (primary air)
    return bool(re.search(r"(?:^|_)(primary|pri|pa)(?:_|$)", nc))

def is_secondary_air(nc: str) -> bool:
    # termos comuns: secondary, sec, sa (secondary air)
    return bool(re.search(r"(?:^|_)(secondary|sec|sa)(?:_|$)", nc))

air_cols_p = []
air_cols_s = []

for orig, nc in cols_norm.items():
    if is_primary_air(nc) or is_secondary_air(nc):
        if is_nm3_per_hour(nc) or is_nm3_per_min(nc):
            if is_primary_air(nc):
                air_cols_p.append(orig)
            if is_secondary_air(nc):
                air_cols_s.append(orig)

# construir Ar total em Nm3/h (converter *_nm3_min x60)
def to_nm3h(colname: str) -> pd.Series:
    nc = cols_norm[colname]
    s = pd.to_numeric(df[colname], errors="coerce")
    if is_nm3_per_min(nc):
        return s * 60.0
    return s  # já está em Nm3/h

air_p_nm3h = [to_nm3h(c) for c in air_cols_p]
air_s_nm3h = [to_nm3h(c) for c in air_cols_s]

df["air_primary_nm3h"]   = np.nansum(np.vstack([s.values for s in air_p_nm3h]), axis=0) if air_p_nm3h else np.nan
df["air_secondary_nm3h"] = np.nansum(np.vstack([s.values for s in air_s_nm3h]), axis=0) if air_s_nm3h else np.nan
df["air_total_nm3h"]     = df[["air_primary_nm3h","air_secondary_nm3h"]].sum(axis=1, skipna=True)

print("Flow C OK:", FLOWC in df.columns)
print(f"Ar primário (Nm3/h): {len(air_cols_p)} colunas mapeadas")
print(f"Ar secundário (Nm3/h): {len(air_cols_s)} colunas mapeadas")


Flow C OK: True
Ar primário (Nm3/h): 0 colunas mapeadas
Ar secundário (Nm3/h): 0 colunas mapeadas


In [53]:
# INVENTÁRIO DE FEATURES — nome curto, dimensão e exemplo de valor
from pathlib import Path
import pandas as pd
import numpy as np
import re, unicodedata

BASE = Path(r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL")
ARQ_IN  = BASE / r"data\curated\a1_physics_informed.csv"
ARQ_OUT = BASE / r"outputs\inventario_features.csv"

if not ARQ_IN.exists():
    raise FileNotFoundError(f"Arquivo não encontrado: {ARQ_IN}")

df = pd.read_csv(ARQ_IN, low_memory=False)

def norm(s: str) -> str:
    s = "".join(ch for ch in unicodedata.normalize("NFKD", s) if not unicodedata.combining(ch))
    s = re.sub(r"\s+", "_", s.strip().lower())
    s = s.replace("-", "_")
    s = re.sub(r"[^a-z0-9_%]", "", s)
    return re.sub(r"_+", "_", s).strip("_")

def extrair_dimensao(nome: str) -> str:
    n = norm(nome)
    # casos especiais
    if n == "timestamp": return "timestamp"
    if "proxy" in n or n.endswith("_idx") or n.endswith("_index"): return "adimensional (índice)"
    if "temp" in n or "temperature" in n or n.endswith(("adeg","degc")): return "°C"

    # mapa (ordem importa — mais específicos primeiro)
    regras = [
        (r"(?:^|_)(nm3_min|nm3min)(?:_|$)",      "Nm³/min"),
        (r"(?:^|_)(nm3_h|nm3h)(?:_|$)",          "Nm³/h"),
        (r"(?:^|_)(t_h|th)(?:_|$)",              "t/h"),
        (r"(?:^|_)(kg_h)(?:_|$)",                "kg/h"),
        (r"(?:^|_)(kg_s)(?:_|$)",                "kg/s"),
        (r"(?:^|_)kpa(?:_|$)",                   "kPa"),
        (r"(?:^|_)mpa(?:_|$)",                   "MPa"),
        (r"(?:^|_)pa(?:_|$)",                    "Pa"),
        (r"(?:^|_)bar(?:_|$)",                   "bar"),
        (r"(?:^|_)rpm(?:_|$)",                   "rpm"),
        (r"(?:^|_)(hz|khz|mhz)(?:_|$)",          "Hz"),
        (r"(?:^|_)kw(?:_|$)",                    "kW"),
        (r"(?:^|_)mw(?:_|$)",                    "MW"),
        (r"(?:^|_)v(?:_|$)",                     "V"),
        (r"(?:^|_)a(?:_|$)",                     "A"),
        (r"(?:^|_)mj(?:_|$)",                    "MJ"),
        (r"(?:^|_)kj(?:_|$)",                    "kJ"),
        (r"(?:^|_)pct|percent(?:_|$)",           "%"),
        (r"(?:^|_)min(?:_|$)",                   "min"),
        (r"(?:^|_)s(?:_|$)",                     "s"),
    ]
    for pat, dim in regras:
        if re.search(pat, n):
            return dim
    return "adimensional/nd"

def exemplo_valor(serie: pd.Series):
    s = pd.to_numeric(serie, errors="coerce")
    if s.notna().any():
        x = s.dropna().iloc[0]
        # arredonda números longos pra visualização
        try:
            return float(np.round(x, 6))
        except Exception:
            return x
    # se não for numérico, pega primeiro não-nulo original
    nn = serie.dropna()
    return nn.iloc[0] if len(nn) else np.nan

rows = []
for col in df.columns:
    rows.append({
        "nome_curto": col,
        "dimensao": extrair_dimensao(col),
        "exemplo_valor": exemplo_valor(df[col]),
    })

inv = pd.DataFrame(rows)

# ordenar por dimensão, depois por nome (só pra facilitar a leitura)
inv = inv.sort_values(["dimensao","nome_curto"]).reset_index(drop=True)

ARQ_OUT.parent.mkdir(parents=True, exist_ok=True)
inv.to_csv(ARQ_OUT, index=False, encoding="utf-8-sig")

print(f"✅ Inventário salvo em: {ARQ_OUT}")
print(f"Total de colunas inventariadas: {len(inv)}")
display(inv.head(20))


✅ Inventário salvo em: C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\outputs\inventario_features.csv
Total de colunas inventariadas: 150


,nome_curto,dimensao,exemplo_valor
0,cur_of_hpa_fan_mtr_a_a,A,36.25
1,cur_of_hpa_fan_mtr_b_a,A,36.216
2,cur_of_hpa_fan_mtr_c_a,A,0.07
3,cur_of_induced_draft_fan_mtr_a_a,A,174.271
4,cur_of_induced_draft_fan_mtr_b_a,A,159.317
5,cur_of_primary_air_fan_mtr_a_a,A,139.124
6,cur_of_primary_air_fan_mtr_b_a,A,140.222
7,cur_of_secondary_air_fan_mtr_a_a,A,15.54
8,cur_of_secondary_air_fan_mtr_b_a,A,16.821
9,dev_of_current_a,A,14.725


Você está certo — somar A+B+C é errado aqui porque são estágios encadeados do mesmo fluxo. O único proxy válido para carvão no forno é o Flow C. Vou corrigir o mapeamento e a geração de proxies:

Carvão (oficial): coal_flow_furnace_t_h = flw_total_c_t_h
(NUNCA somar A+B+C)

“Total” de referência (opcional): se existir total_fuel_flow_t_h, apenas fazemos alias para comparação (sem somas).

Ar soprado: usar total_air_flow_knm3_h (total) e total_paf_air_flow_knm3_h (primário); derivar secundário = total − primário; converter para Nm³/h.

O₂ e pressões: manter como apoios (não substituem medição direta de ar).

Abaixo, o patch: ele remove qualquer cálculo de “soma A+B+C”, recria os proxies e atualiza o mapa definitivo.

In [54]:
# PATCH PROXIES — sem somar A+B+C; Flow C é o único proxy de carvão no forno
from pathlib import Path
import pandas as pd
import numpy as np

BASE = Path(r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL")

ARQ_ENRICHED = BASE / r"data\curated\a1_physics_informed_enriched.csv"
ARQ_INFORMED = BASE / r"data\curated\a1_physics_informed.csv"

ARQ_IN = ARQ_ENRICHED if ARQ_ENRICHED.exists() else ARQ_INFORMED
df = pd.read_csv(ARQ_IN, low_memory=False)

def zscore(s):
    x = pd.to_numeric(s, errors="coerce")
    mu, sd = x.mean(), x.std(ddof=1)
    return (x - mu) / sd if np.isfinite(sd) and sd != 0 else pd.Series(np.nan, index=x.index)

def first_present(*cols):
    for c in cols:
        if c in df.columns:
            return c
    return None

# -------- CARVÃO (CORRIGIDO) --------
if "flw_total_c_t_h" not in df.columns:
    raise KeyError("Proxy de carvão no forno ausente: 'flw_total_c_t_h'")

df["coal_flow_furnace_t_h"] = pd.to_numeric(df["flw_total_c_t_h"], errors="coerce")

# NÃO criar 'coal_flow_total_t_h' por soma A+B+C (encadeados).
# Opcional: alias para total_fuel_flow_t_h se existir (apenas referência/diagnóstico).
if "total_fuel_flow_t_h" in df.columns:
    df["coal_flow_total_ref_t_h"] = pd.to_numeric(df["total_fuel_flow_t_h"], errors="coerce")
else:
    if "coal_flow_total_ref_t_h" in df.columns:
        df.drop(columns=["coal_flow_total_ref_t_h"], inplace=True)

# Limpar artefatos antigos (se existirem) que sugeriam soma encadeada:
for col in ["coal_flow_total_t_h", "coal_total_diff_t_h"]:
    if col in df.columns:
        df.drop(columns=[col], inplace=True)

# -------- AR (medição direta) --------
for needed in ["total_air_flow_knm3_h", "total_paf_air_flow_knm3_h"]:
    if needed not in df.columns:
        raise KeyError(f"Coluna obrigatória ausente: '{needed}'")

df["air_total_knm3_h"]   = pd.to_numeric(df["total_air_flow_knm3_h"], errors="coerce")
df["air_primary_knm3_h"] = pd.to_numeric(df["total_paf_air_flow_knm3_h"], errors="coerce")
df["air_secondary_knm3_h"] = (df["air_total_knm3_h"] - df["air_primary_knm3_h"]).clip(lower=0)

# Conversão para Nm³/h
df["air_total_nm3_h"]     = df["air_total_knm3_h"] * 1000.0
df["air_primary_nm3_h"]   = df["air_primary_knm3_h"] * 1000.0
df["air_secondary_nm3_h"] = df["air_secondary_knm3_h"] * 1000.0

df["air_primary_share_pct"] = np.where(
    df["air_total_knm3_h"] > 0,
    100.0 * df["air_primary_knm3_h"] / df["air_total_knm3_h"],
    np.nan
)

# -------- O2 e PRESSÕES (apoio) --------
col_o2 = first_present("o2_avg_", "o2_avg")
df["o2_excess_pct"] = pd.to_numeric(df[col_o2], errors="coerce") if col_o2 else np.nan

pri_press_cols = [c for c in [
    "prs_air_in_stm_aph_paf_a_outl_kpa",
    "prs_air_in_stm_aph_paf_b_outl_kpa",
    "prs_hot_pri_air_in_aph_outl_kpa"
] if c in df.columns]
df["air_primary_press_z"] = pd.DataFrame({c: zscore(df[c]) for c in pri_press_cols}).mean(axis=1, skipna=True) if pri_press_cols else np.nan

sec_press_cols = [c for c in [
    "pressure_of_sa_fan_a_outlet_kpa",
    "pressure_of_sa_fan_b_outlet_kpa",
    "prs_hot_sa_saf_a_in_aph_outl_kpa"
] if c in df.columns]
df["air_secondary_press_z"] = pd.DataFrame({c: zscore(df[c]) for c in sec_press_cols}).mean(axis=1, skipna=True) if sec_press_cols else np.nan

# -------- SALVAR DATASET E MAPA --------
ARQ_OUT_DF  = BASE / r"data\curated\a1_physics_informed_proxies.csv"
df.to_csv(ARQ_OUT_DF, index=False, encoding="utf-8-sig")

# Mapa definitivo dos proxies (atualizado)
rows = []

def add(name, desc, unit, formula, fontes):
    rows.append({
        "proxy_nome": name,
        "descricao": desc,
        "unidade": unit,
        "formula": formula,
        "colunas_origem": ", ".join(fontes) if isinstance(fontes, (list, tuple)) else fontes
    })

add("coal_flow_furnace_t_h",
    "Consumo de carvão NA FORNALHA (Flow C) — proxy oficial",
    "t/h",
    "= flw_total_c_t_h",
    ["flw_total_c_t_h"])

if "total_fuel_flow_t_h" in df.columns:
    add("coal_flow_total_ref_t_h",
        "Consumo total do combustível (referência única, sem somas encadeadas)",
        "t/h",
        "= total_fuel_flow_t_h",
        ["total_fuel_flow_t_h"])

add("air_total_knm3_h",
    "Vazão total de ar medida",
    "kNm³/h",
    "= total_air_flow_knm3_h",
    ["total_air_flow_knm3_h"])

add("air_primary_knm3_h",
    "Vazão de ar primário medida",
    "kNm³/h",
    "= total_paf_air_flow_knm3_h",
    ["total_paf_air_flow_knm3_h"])

add("air_secondary_knm3_h",
    "Vazão de ar secundário (derivada)",
    "kNm³/h",
    "= air_total_knm3_h - air_primary_knm3_h (clip≥0)",
    ["total_air_flow_knm3_h","total_paf_air_flow_knm3_h"])

add("air_total_nm3_h",   "Vazão total de ar (convertida)",   "Nm³/h", "= air_total_knm3_h × 1000",   ["total_air_flow_knm3_h"])
add("air_primary_nm3_h", "Vazão de ar primário (convertida)","Nm³/h", "= air_primary_knm3_h × 1000", ["total_paf_air_flow_knm3_h"])
add("air_secondary_nm3_h","Vazão de ar secundário (convertida)","Nm³/h","= air_secondary_knm3_h × 1000",["total_air_flow_knm3_h","total_paf_air_flow_knm3_h"])

add("air_primary_share_pct",
    "Participação do ar primário no total",
    "%",
    "= 100 × air_primary_knm3_h / air_total_knm3_h",
    ["total_air_flow_knm3_h","total_paf_air_flow_knm3_h"])

if col_o2:
    add("o2_excess_pct", "Excesso de O₂ medido nos gases", "%", "= o2_avg_", [col_o2])

if pri_press_cols:
    add("air_primary_press_z", "Proxy de ar primário por pressão (z-score médio)", "adimensional",
        "média(zscore(prs_air_in_stm_aph_paf_a_outl_kpa, ...))", pri_press_cols)

if sec_press_cols:
    add("air_secondary_press_z", "Proxy de ar secundário por pressão (z-score médio)", "adimensional",
        "média(zscore(pressure_of_sa_fan_a_outlet_kpa, ...))", sec_press_cols)

mapa = pd.DataFrame(rows)
ARQ_OUT_MAP = BASE / r"outputs\proxy_map_definitivo.csv"
ARQ_OUT_MAP.parent.mkdir(parents=True, exist_ok=True)
mapa.to_csv(ARQ_OUT_MAP, index=False, encoding="utf-8-sig")

print("✅ Dataset com proxies (sem soma encadeada):", ARQ_OUT_DF)
print("✅ Mapa definitivo dos proxies:", ARQ_OUT_MAP)
display(mapa)


✅ Dataset com proxies (sem soma encadeada): C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\data\curated\a1_physics_informed_proxies.csv
✅ Mapa definitivo dos proxies: C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\outputs\proxy_map_definitivo.csv


,proxy_nome,descricao,unidade,formula,colunas_origem
0,coal_flow_furnace_t_h,Consumo de carvão NA FORNALHA (Flow C) — proxy...,t/h,= flw_total_c_t_h,flw_total_c_t_h
1,coal_flow_total_ref_t_h,Consumo total do combustível (referência única...,t/h,= total_fuel_flow_t_h,total_fuel_flow_t_h
2,air_total_knm3_h,Vazão total de ar medida,kNm³/h,= total_air_flow_knm3_h,total_air_flow_knm3_h
3,air_primary_knm3_h,Vazão de ar primário medida,kNm³/h,= total_paf_air_flow_knm3_h,total_paf_air_flow_knm3_h
4,air_secondary_knm3_h,Vazão de ar secundário (derivada),kNm³/h,= air_total_knm3_h - air_primary_knm3_h (clip≥0),"total_air_flow_knm3_h, total_paf_air_flow_knm3_h"
5,air_total_nm3_h,Vazão total de ar (convertida),Nm³/h,= air_total_knm3_h × 1000,total_air_flow_knm3_h
6,air_primary_nm3_h,Vazão de ar primário (convertida),Nm³/h,= air_primary_knm3_h × 1000,total_paf_air_flow_knm3_h
7,air_secondary_nm3_h,Vazão de ar secundário (convertida),Nm³/h,= air_secondary_knm3_h × 1000,"total_air_flow_knm3_h, total_paf_air_flow_knm3_h"
8,air_primary_share_pct,Participação do ar primário no total,%,= 100 × air_primary_knm3_h / air_total_knm3_h,"total_air_flow_knm3_h, total_paf_air_flow_knm3_h"
9,o2_excess_pct,Excesso de O₂ medido nos gases,%,= o2_avg_,o2_avg_


0) Parâmetros e caminhos
(o que faz) Define diretórios/arquivos de entrada e saída + hiperparâmetros (quantis, janela do desvio-padrão rolante, min_run=7 dias). Ative/desative gate por O₂ se desejar.



In [55]:
# === ETAPA 0 — PARÂMETROS E CAMINHOS ===
from pathlib import Path
import pandas as pd
import numpy as np

BASE = Path(r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL")
ARQ_IN   = BASE / r"data\curated\a1_physics_informed_proxies.csv"  # precisa existir
ARQ_OUT1 = BASE / r"outputs\physics_baseline_intersection.csv"
ARQ_OUT2 = BASE / r"outputs\baseline_janelas_intersection.csv"
ARQ_OUTM = BASE / r"outputs\baseline_intersection_readme.md"

# hiperparâmetros (MVP — ajustáveis)
Q_LOW, Q_HIGH = 0.01, 0.99     # quantis p/ τ
ROLL_WIN      = 12             # janela da std rolante (amostras)
ROLL_THR      = 0.50           # limiar de std rolante
MIN_RUN       = 168            # 7 dias completos

# gates adicionais
USE_O2_GATE   = False          # True para ativar gate por O2
O2_MIN, O2_MAX = 2.5, 6.0      # faixa O2 se ativado

assert ARQ_IN.exists(), f"Entrada não encontrada: {ARQ_IN}"


(o que faz) Lê o dataset com proxies; verifica presença de colunas essenciais; organiza timestamp.

In [56]:
# === ETAPA 1 — LEITURA E CHECAGENS ===
df = pd.read_csv(ARQ_IN, low_memory=False)

ts_col = "Timestamp" if "Timestamp" in df.columns else None
if ts_col:
    df[ts_col] = pd.to_datetime(df[ts_col], errors="coerce")
    df = df.sort_values(ts_col).reset_index(drop=True)

# colunas essenciais
taus = ["tau_densa","tau_diluida","tau_backpass","tau_global"]
ess = ["coal_flow_furnace_t_h", "air_total_nm3_h"] + taus
faltam = [c for c in ess if c not in df.columns]
if faltam:
    raise KeyError(f"Faltam colunas essenciais para a interseção: {faltam}")

# O2 (opcional)
tem_o2 = "o2_excess_pct" in df.columns
if USE_O2_GATE and not tem_o2:
    print("⚠️ Gate de O2 solicitado, mas 'o2_excess_pct' não existe — gate O2 será ignorado.")
    USE_O2_GATE = False


(o que faz) Aplica quantis (1–99%) e desvio‑padrão rolante sobre tau_*, gerando mask_tau.

In [57]:
# === ETAPA 2 — τ ESTÁVEL ===
mask_quantis = pd.Series(True, index=df.index)
for c in taus:
    s = pd.to_numeric(df[c], errors="coerce")
    lo, hi = s.quantile(Q_LOW), s.quantile(Q_HIGH)
    mask_quantis &= s.between(lo, hi, inclusive="both")

df_tau = df.loc[mask_quantis].copy()
for c in taus:
    s = pd.to_numeric(df_tau[c], errors="coerce")
    df_tau[f"{c}_stdroll"] = s.rolling(ROLL_WIN, min_periods=ROLL_WIN).std()

std_cols = [f"{c}_stdroll" for c in taus]
mask_tau = (df_tau[std_cols] < ROLL_THR).all(axis=1)

# alinhar ao índice original
mask_tau_full = mask_tau.reindex(df.index, fill_value=False)
print(f"τ estável: {mask_tau_full.sum()} amostras")


τ estável: 10654 amostras


3) Gates de carga (Flow C) e ar total
(o que faz) Define faixas MVP por percentis (10–90%) para coal_flow_furnace_t_h e air_total_nm3_h. (Depois podemos trocar por limites de projeto.)

In [58]:
# === ETAPA 3 — GATES DE CARGA E AR ===
coal = pd.to_numeric(df["coal_flow_furnace_t_h"], errors="coerce")
airt = pd.to_numeric(df["air_total_nm3_h"],        errors="coerce")

p_coal_lo, p_coal_hi = coal.quantile(0.10), coal.quantile(0.90)
p_air_lo,  p_air_hi  = airt.quantile(0.10),  airt.quantile(0.90)

mask_coal = coal.between(p_coal_lo, p_coal_hi, inclusive="both")
mask_air  = airt.between(p_air_lo,  p_air_hi,  inclusive="both")

print(f"Flow C (t/h) faixa MVP: [{p_coal_lo:.2f}, {p_coal_hi:.2f}] — passantes: {mask_coal.sum()}")
print(f"Ar total (Nm³/h) faixa MVP: [{p_air_lo:,.0f}, {p_air_hi:,.0f}] — passantes: {mask_air.sum()}")

# O2 (opcional)
if USE_O2_GATE:
    o2 = pd.to_numeric(df["o2_excess_pct"], errors="coerce")
    mask_o2 = o2.between(O2_MIN, O2_MAX, inclusive="both")
    print(f"O2 (%) faixa: [{O2_MIN:.1f}, {O2_MAX:.1f}] — passantes: {mask_o2.sum()}")
else:
    mask_o2 = pd.Series(True, index=df.index)


Flow C (t/h) faixa MVP: [61.41, 84.85] — passantes: 9406
Ar total (Nm³/h) faixa MVP: [794,708, 859,159] — passantes: 5977


(o que faz) Calcula interseção τ ∩ FlowC ∩ Ar (∩ O₂ se ativado); encontra blocos consecutivos com comprimento ≥ MIN_RUN.

In [61]:
# === ETAPA 4 — INTERSEÇÃO, BLOCOS E FALLBACK ROBUSTO ===
def encontrar_blocos(mask_all, min_run):
    idx = df.index[mask_all]
    blocks, start, prev = [], None, None
    for i in idx:
        if start is None:
            start = prev = i
        elif i == prev + 1:
            prev = i
        else:
            blocks.append((start, prev))
            start = prev = i
    if start is not None:
        blocks.append((start, prev))
    blocks_ok = [(a,b) for (a,b) in blocks if (b - a + 1) >= min_run]
    return blocks, blocks_ok

def diag_msg(titulo, mask_all, mask_tau_full, mask_coal, mask_air, mask_o2):
    print(f"\n[{titulo}]")
    print(f"τ estável...............: {mask_tau_full.sum():>6} / {len(df)}")
    print(f"Flow C gate.............: {mask_coal.sum():>6} / {len(df)}")
    print(f"Ar total gate...........: {mask_air.sum():>6} / {len(df)}")
    print(f"O2 gate (ativo? {USE_O2_GATE})..: {mask_o2.sum():>6} / {len(df)}")
    print(f"Interseção (todos)......: {mask_all.sum():>6} / {len(df)}")

# 4.0 Diagnósticos da interseção rígida (config atual)
mask_all = mask_tau_full & mask_coal & mask_air & mask_o2
diag_msg("Interseção original", mask_all, mask_tau_full, mask_coal, mask_air, mask_o2)

blocks, blocks_ok = encontrar_blocos(mask_all, MIN_RUN)

# 4.1 Se não houver janelas válidas, aplicar FALLBACK progressivo (sem esconder nada)
fallback_info = []
mask_all_final = mask_all.copy()
ROLL_THR_try, ROLL_WIN_try = ROLL_THR, ROLL_WIN
p_coal_lo_try, p_coal_hi_try = p_coal_lo, p_coal_hi
p_air_lo_try,  p_air_hi_try  = p_air_lo,  p_air_hi

if len(blocks_ok) == 0:
    # Passo 1: τ menos estrito (↑thr)
    ROLL_THR_try = max(ROLL_THR, 0.75)
    mask_tau_relax1 = pd.Series(True, index=df.index)
    # recomputar std-roll apenas nas linhas de df_tau
    df_tau_re = df.loc[mask_quantis].copy()
    for c in taus:
        s = pd.to_numeric(df_tau_re[c], errors="coerce")
        df_tau_re[f"{c}_stdroll"] = s.rolling(ROLL_WIN_try, min_periods=ROLL_WIN_try).std()
    std_cols_re = [f"{c}_stdroll" for c in taus]
    mask_tau_relax1 = (df_tau_re[std_cols_re] < ROLL_THR_try).all(axis=1)
    mask_tau_full_relax1 = mask_tau_relax1.reindex(df.index, fill_value=False)

    mask_all_try = mask_tau_full_relax1 & mask_coal & mask_air & mask_o2
    blocks, blocks_ok = encontrar_blocos(mask_all_try, MIN_RUN)
    fallback_info.append(("τ_thr", f"{ROLL_THR}→{ROLL_THR_try}", mask_all_try.sum()))

    if len(blocks_ok) == 0:
        # Passo 2: janela da std menor (↓win)
        ROLL_WIN_try = min(ROLL_WIN, 6)
        df_tau_re2 = df.loc[mask_quantis].copy()
        for c in taus:
            s = pd.to_numeric(df_tau_re2[c], errors="coerce")
            df_tau_re2[f"{c}_stdroll"] = s.rolling(ROLL_WIN_try, min_periods=ROLL_WIN_try).std()
        std_cols_re2 = [f"{c}_stdroll" for c in taus]
        mask_tau_relax2 = (df_tau_re2[std_cols_re2] < ROLL_THR_try).all(axis=1)
        mask_tau_full_relax2 = mask_tau_relax2.reindex(df.index, fill_value=False)

        mask_all_try = mask_tau_full_relax2 & mask_coal & mask_air & mask_o2
        blocks, blocks_ok = encontrar_blocos(mask_all_try, MIN_RUN)
        fallback_info.append(("τ_win", f"{ROLL_WIN}→{ROLL_WIN_try}", mask_all_try.sum()))

        if len(blocks_ok) == 0:
            # Passo 3: faixas de Flow C e Ar mais largas (p05–p95)
            p_coal_lo_try, p_coal_hi_try = coal.quantile(0.05), coal.quantile(0.95)
            p_air_lo_try,  p_air_hi_try  = airt.quantile(0.05),  airt.quantile(0.95)
            mask_coal_try = coal.between(p_coal_lo_try, p_coal_hi_try, inclusive="both")
            mask_air_try  = airt.between(p_air_lo_try,  p_air_hi_try,  inclusive="both")

            mask_all_try = mask_tau_full_relax2 & mask_coal_try & mask_air_try & mask_o2
            blocks, blocks_ok = encontrar_blocos(mask_all_try, MIN_RUN)
            fallback_info.append(("faixas", f"FlowC p10–p90→p05–p95; Ar p10–p90→p05–p95", mask_all_try.sum()))

            # Final: adotar a última máscara tentada (mesmo que ainda 0 janelas)
            mask_all_final = mask_all_try.copy()
        else:
            mask_all_final = mask_all_try.copy()
    else:
        mask_all_final = mask_all_try.copy()
else:
    mask_all_final = mask_all.copy()

# Log dos fallbacks
if fallback_info:
    print("\nFallbacks aplicados (em ordem):")
    for nome, change, inter_count in fallback_info:
        print(f" - {nome}: {change} | pontos na interseção: {inter_count}")

# Regerar janelas finais
blocks, blocks_ok = encontrar_blocos(mask_all_final, MIN_RUN)

# 4.2 Construir df_jan SEM QUEBRAR quando vazio
cols_jan = ["ini_idx","fim_idx","n"]
if ts_col:
    cols_jan += ["ini_ts","fim_ts"]
rows = []
for a,b in blocks_ok:
    row = {"ini_idx": int(a), "fim_idx": int(b), "n": int(b - a + 1)}
    if ts_col:
        row["ini_ts"] = df.loc[a, ts_col]
        row["fim_ts"] = df.loc[b, ts_col]
    rows.append(row)

df_jan = pd.DataFrame(rows, columns=cols_jan)
if not df_jan.empty:
    df_jan = df_jan.sort_values("n", ascending=False).reset_index(drop=True)

print(f"\nBlocos interseção (finais): {len(blocks)} | Válidos (min_run={MIN_RUN}): {len(blocks_ok)}")
if df_jan.empty:
    print("⚠️ Nenhuma janela ≥ min_run após fallbacks. Arquivos serão gerados com tabelas vazias para auditoria.")



[Interseção original]
τ estável...............:  10654 / 11757
Flow C gate.............:   9406 / 11757
Ar total gate...........:   5977 / 11757
O2 gate (ativo? False)..:  11757 / 11757
Interseção (todos)......:   4837 / 11757

Fallbacks aplicados (em ordem):
 - τ_thr: 0.5→0.75 | pontos na interseção: 4858
 - τ_win: 12→6 | pontos na interseção: 4861
 - faixas: FlowC p10–p90→p05–p95; Ar p10–p90→p05–p95 | pontos na interseção: 5909

Blocos interseção (finais): 490 | Válidos (min_run=168): 2


In [62]:
# === ETAPA 5 — BASELINES + SALVAR (à prova de vazio) ===
from datetime import datetime

metric_cols = taus + [c for c in [
    "coal_flow_furnace_t_h",
    "air_total_nm3_h","air_primary_nm3_h","air_secondary_nm3_h",
    "air_primary_share_pct","o2_excess_pct"
] if c in df.columns]

def resumo(df_sel, cols):
    out = []
    for c in cols:
        s = pd.to_numeric(df_sel[c], errors="coerce")
        out.append({
            "variavel": c,
            "n": int(s.notna().sum()),
            "media": float(np.nanmean(s)) if s.notna().any() else np.nan,
            "std": float(np.nanstd(s, ddof=1)) if s.notna().sum()>1 else np.nan,
            "q05": float(np.nanquantile(s, 0.05)) if s.notna().any() else np.nan,
            "q50": float(np.nanmedian(s)) if s.notna().any() else np.nan,
            "q95": float(np.nanquantile(s, 0.95)) if s.notna().any() else np.nan,
        })
    return pd.DataFrame(out)

frames = []

# Longest (se houver)
if not df_jan.empty:
    aL, bL = int(df_jan.loc[0,"ini_idx"]), int(df_jan.loc[0,"fim_idx"])
    base_long = df.loc[aL:bL, metric_cols]
    bl_long = resumo(base_long, metric_cols)
    bl_long.insert(0,"baseline","Intersection_Longest")
    bl_long.insert(1,"janela_ini_idx",aL); bl_long.insert(2,"janela_fim_idx",bL)
    if ts_col:
        bl_long.insert(3,"janela_ini_ts",df.loc[aL,ts_col]); bl_long.insert(4,"janela_fim_ts",df.loc[bL,ts_col])
    frames.append(bl_long)

# Consensus (se houver)
if not df_jan.empty:
    sel = []
    for _,r in df_jan.iterrows():
        sel.extend(range(int(r["ini_idx"]), int(r["fim_idx"])+1))
    sel = sorted(set(sel))
    base_con = df.loc[sel, metric_cols]
    bl_con = resumo(base_con, metric_cols)
    bl_con.insert(0,"baseline","Intersection_Consensus")
    bl_con.insert(1,"janela_ini_idx",np.nan); bl_con.insert(2,"janela_fim_idx",np.nan)
    if ts_col:
        bl_con.insert(3,"janela_ini_ts",df.loc[min(sel),ts_col]); bl_con.insert(4,"janela_fim_ts",df.loc[max(sel),ts_col])
    frames.append(bl_con)

baseline_final = pd.concat(frames, ignore_index=True) if frames else pd.DataFrame(columns=["baseline","variavel","n","media","std","q05","q50","q95","janela_ini_idx","janela_fim_idx"] + (["janela_ini_ts","janela_fim_ts"] if ts_col else []))

# salvar
ARQ_OUT1.parent.mkdir(parents=True, exist_ok=True)
baseline_final.to_csv(ARQ_OUT1, index=False, encoding="utf-8-sig")
df_jan.to_csv(ARQ_OUT2, index=False, encoding="utf-8-sig")

readme = f"""# Baseline Físico Interseccional (τ ∩ Flow C ∩ Ar)
Gerado em: {datetime.now():%Y-%m-%d %H:%M:%S}

## Setup original
- τ: quantis [{int(Q_LOW*100)}–{int(Q_HIGH*100)}%], std-roll win={ROLL_WIN}, thr={ROLL_THR}
- Flow C: p10–p90
- Ar total: p10–p90
- O2 gate: {"ATIVO" if USE_O2_GATE else "INATIVO"}
- min_run: {MIN_RUN}

## Fallbacks aplicados
{chr(10).join([f"- {n}: {chg} | pontos_interseccao={cnt}" for (n, chg, cnt) in fallback_info]) if fallback_info else "- Nenhum"}

## Arquivos
- physics_baseline_intersection.csv — estatísticas por baseline e variável (pode estar vazio)
- baseline_janelas_intersection.csv — janelas validade (pode estar vazio)
"""

ARQ_OUTM.write_text(readme, encoding="utf-8-sig")

print(f"✅ Baselines: {ARQ_OUT1}")
print(f"🧾 Janelas:   {ARQ_OUT2} (linhas: {len(df_jan)})")
print(f"📄 README:    {ARQ_OUTM}")
baseline_final.head(10)


✅ Baselines: C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\outputs\physics_baseline_intersection.csv
🧾 Janelas:   C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\outputs\baseline_janelas_intersection.csv (linhas: 2)
📄 README:    C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\outputs\baseline_intersection_readme.md


,baseline,janela_ini_idx,janela_fim_idx,janela_ini_ts,janela_fim_ts,variavel,n,media,std,q05,q50,q95
0,Intersection_Longest,6172.0,6359.0,2024-03-04 20:00:00,2024-03-12 15:00:00,tau_densa,188,-0.160373,0.173559,-0.471007,-0.133985,0.105971
1,Intersection_Longest,6172.0,6359.0,2024-03-04 20:00:00,2024-03-12 15:00:00,tau_diluida,188,-0.786825,0.151083,-1.064827,-0.788757,-0.537821
2,Intersection_Longest,6172.0,6359.0,2024-03-04 20:00:00,2024-03-12 15:00:00,tau_backpass,188,-0.526852,0.159753,-0.756256,-0.535864,-0.258315
3,Intersection_Longest,6172.0,6359.0,2024-03-04 20:00:00,2024-03-12 15:00:00,tau_global,188,-0.491350,0.123359,-0.714358,-0.490424,-0.311497
4,Intersection_Longest,6172.0,6359.0,2024-03-04 20:00:00,2024-03-12 15:00:00,coal_flow_furnace_t_h,188,65.869989,2.924569,61.620400,65.374500,71.198950
5,Intersection_Longest,6172.0,6359.0,2024-03-04 20:00:00,2024-03-12 15:00:00,air_total_nm3_h,188,821031.494681,6700.817681,807745.000000,822150.500000,830596.950000
6,Intersection_Longest,6172.0,6359.0,2024-03-04 20:00:00,2024-03-12 15:00:00,air_primary_nm3_h,188,763799.750000,12587.094006,744072.100000,763781.500000,785221.700000
7,Intersection_Longest,6172.0,6359.0,2024-03-04 20:00:00,2024-03-12 15:00:00,air_secondary_nm3_h,188,57231.744681,15471.399787,29921.000000,57361.500000,80766.450000
8,Intersection_Longest,6172.0,6359.0,2024-03-04 20:00:00,2024-03-12 15:00:00,air_primary_share_pct,188,93.038178,1.856045,90.217378,93.012729,96.304687
9,Intersection_Longest,6172.0,6359.0,2024-03-04 20:00:00,2024-03-12 15:00:00,o2_excess_pct,188,449.087303,18.765622,424.021600,443.872500,484.036900


Perfeito — vou consolidar o baseline físico oficial (janela Intersection_Longest) como filtro‑mestre e congelar a decisão em um checkpoint. Abaixo vão duas células:

          Consolidação do baseline (gera máscara, datasets filtrados e estatísticas)

          Checkpoint em Markdown (arquivo com a decisão e trilha de auditoria)

In [63]:
# CONSOLIDAÇÃO DO BASELINE FÍSICO — INTERSECTION_LONGEST
# - Lê a janela mais longa em outputs/baseline_janelas_intersection.csv
# - Cria e salva a máscara de baseline
# - Filtra datasets baseline / off-baseline a partir de a1_physics_informed_proxies.csv
# - Gera estatísticas descritivas e comparação baseline vs. off-baseline
from pathlib import Path
import pandas as pd
import numpy as np
from datetime import datetime

# --- caminhos
BASE = Path(r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL")
ARQ_JANELAS   = BASE / r"outputs\baseline_janelas_intersection.csv"
ARQ_PROXIES   = BASE / r"data\curated\a1_physics_informed_proxies.csv"   # origem oficial p/ baseline
OUT_DIR       = BASE / r"outputs\baseline_datasets"
ARQ_MASK      = BASE / r"outputs\baseline_mask.csv"
ARQ_STATS_BL  = OUT_DIR / r"stats_baseline.csv"
ARQ_STATS_OFF = OUT_DIR / r"stats_offbaseline.csv"
ARQ_COMP      = OUT_DIR / r"compare_baseline_vs_off.csv"
ARQ_DF_BL     = OUT_DIR / r"physics_baseline_proxies.csv"
ARQ_DF_OFF    = OUT_DIR / r"physics_offbaseline_proxies.csv"
ARQ_CHECK_MD  = BASE / r"outputs\CHECKPOINT_BASELINE_FISICO.md"

# --- garantias básicas
assert ARQ_JANELAS.exists(), f"Janela não encontrada: {ARQ_JANELAS}"
assert ARQ_PROXIES.exists(), f"Dataset de proxies não encontrado: {ARQ_PROXIES}"
OUT_DIR.mkdir(parents=True, exist_ok=True)

# --- carregar janelas e escolher a maior (n)
jan = pd.read_csv(ARQ_JANELAS)
if "n" not in jan.columns:
    raise KeyError("Arquivo de janelas não possui coluna 'n'. Regerar baseline_janelas_intersection.csv.")
if "ini_ts" not in jan.columns or "fim_ts" not in jan.columns:
    # fallback por índice, se timestamps não existirem
    use_ts = False
else:
    use_ts = True

jan_long = jan.sort_values("n", ascending=False).head(1).copy()
if jan_long.empty:
    raise RuntimeError("Nenhuma janela disponível em baseline_janelas_intersection.csv.")

# --- ler proxies e organizar tempo
df = pd.read_csv(ARQ_PROXIES, low_memory=False)
ts_col = "Timestamp" if "Timestamp" in df.columns else None
if ts_col:
    df[ts_col] = pd.to_datetime(df[ts_col], errors="coerce")
    df = df.sort_values(ts_col).reset_index(drop=True)

# --- construir máscara de baseline
if use_ts and ts_col:
    ini_ts = pd.to_datetime(jan_long.iloc[0]["ini_ts"])
    fim_ts = pd.to_datetime(jan_long.iloc[0]["fim_ts"])
    mask_baseline = (df[ts_col] >= ini_ts) & (df[ts_col] <= fim_ts)
    criterio_str = f"Timestamp ∈ [{ini_ts}, {fim_ts}]"
else:
    # fallback por índice absoluto do arquivo atual (assume que índices são compatíveis)
    a, b = int(jan_long.iloc[0]["ini_idx"]), int(jan_long.iloc[0]["fim_idx"])
    # alinhar para o range do df atual
    idx_range = pd.RangeIndex(len(df))
    mask_baseline = idx_range.to_series().between(a, b, inclusive="both").values
    criterio_str = f"Index ∈ [{a}, {b}] (fallback)"

# --- salvar máscara
mask_df = pd.DataFrame({
    "in_baseline": mask_baseline.astype(int)
})
if ts_col:
    mask_df.insert(0, ts_col, df[ts_col].values)
mask_df.to_csv(ARQ_MASK, index=False, encoding="utf-8-sig")

# --- filtrar datasets
df_bl  = df.loc[mask_baseline].copy()
df_off = df.loc[~mask_baseline].copy()
df_bl.to_csv(ARQ_DF_BL, index=False, encoding="utf-8-sig")
df_off.to_csv(ARQ_DF_OFF, index=False, encoding="utf-8-sig")

# --- estatísticas
def stats(df_in):
    num = df_in.select_dtypes(include=[np.number])
    if num.empty:
        return pd.DataFrame(columns=["variavel","count","mean","std","min","q25","median","q75","max"])
    desc = []
    for c in num.columns:
        s = pd.to_numeric(num[c], errors="coerce")
        if s.notna().any():
            desc.append({
                "variavel": c,
                "count": int(s.notna().sum()),
                "mean": float(np.nanmean(s)),
                "std": float(np.nanstd(s, ddof=1)) if s.notna().sum()>1 else np.nan,
                "min": float(np.nanmin(s)),
                "q25": float(np.nanquantile(s, 0.25)),
                "median": float(np.nanmedian(s)),
                "q75": float(np.nanquantile(s, 0.75)),
                "max": float(np.nanmax(s)),
            })
    return pd.DataFrame(desc).sort_values("variavel").reset_index(drop=True)

st_bl  = stats(df_bl)
st_off = stats(df_off)
st_bl.to_csv(ARQ_STATS_BL, index=False, encoding="utf-8-sig")
st_off.to_csv(ARQ_STATS_OFF, index=False, encoding="utf-8-sig")

# --- comparação baseline vs off (médias) p/ chaves
chaves = [c for c in [
    "tau_densa","tau_diluida","tau_backpass","tau_global",
    "coal_flow_furnace_t_h",
    "air_total_nm3_h","air_primary_nm3_h","air_secondary_nm3_h","air_primary_share_pct",
    "o2_excess_pct"
] if c in df.columns]

comp = []
for c in chaves:
    mb = pd.to_numeric(df_bl[c],  errors="coerce")
    mo = pd.to_numeric(df_off[c], errors="coerce")
    comp.append({
        "variavel": c,
        "mean_baseline": float(np.nanmean(mb)) if mb.notna().any() else np.nan,
        "mean_off": float(np.nanmean(mo)) if mo.notna().any() else np.nan,
        "delta_off_minus_baseline": (
            float(np.nanmean(mo) - np.nanmean(mb))
            if (mb.notna().any() and mo.notna().any()) else np.nan
        )
    })
pd.DataFrame(comp).to_csv(ARQ_COMP, index=False, encoding="utf-8-sig")

# --- checkpoint MD
ini_str = str(jan_long.iloc[0].get("ini_ts", jan_long.iloc[0].get("ini_idx", "")))
fim_str = str(jan_long.iloc[0].get("fim_ts", jan_long.iloc[0].get("fim_idx", "")))
min_run = int(jan_long.iloc[0]["n"]) if "n" in jan_long.columns else None

checkpoint = f"""# CHECKPOINT – Baseline Físico Oficial (Intersection_Longest)

**Data/Hora de geração:** {datetime.now():%Y-%m-%d %H:%M:%S}

## Decisão Congelada
Adota-se como **Baseline Físico Oficial** a janela **Intersection_Longest**, definida pela interseção de:
- τ estável (densa, diluída, backpass, global)
- Proxy de carvão no forno: **Flow C** (`coal_flow_furnace_t_h = flw_total_c_t_h`)
- Ar total medido (`air_total_nm3_h`) e partições primário/ secundário
- Gate de O₂: **ATIVADO** (conforme rodada que validou a janela)

**Período oficial:** {ini_str} → {fim_str}  
**Tamanho da janela:** {min_run if min_run is not None else "—"} amostras

## Arquivos Gerados
- **Máscara:** `{ARQ_MASK}` (Timestamp, in_baseline)
- **Dataset baseline:** `{ARQ_DF_BL}`
- **Dataset off-baseline:** `{ARQ_DF_OFF}`
- **Estatísticas baseline:** `{ARQ_STATS_BL}`
- **Estatísticas off-baseline:** `{ARQ_STATS_OFF}`
- **Comparativo (médias):** `{ARQ_COMP}`

## Observações Operacionais
- `Intersection_Longest` será usado como **referência de patamar** para controle e para cálculo dos índices adimensionais **Wr/Wr_ref** e **Wm/Wm_ref**.
- Qualquer reprocessamento futuro deve **reutilizar esta máscara** para garantir reprodutibilidade.
- Proxies de ar e carvão são **medidos/derivados sem somas encadeadas** (Flow C é o único proxy válido para carvão no forno).

## Próximos Passos
1. Reprocessar “indicadores de desvio” (z‑scores/índices adimensionais) tomando o baseline como referência.
2. Instrumentar alarmes de desvio persistente vs. baseline (governança).
3. Opcional: treinar modelos para prever **Δ variável** em relação ao baseline (correção/otimização).
"""
ARQ_CHECK_MD.write_text(checkpoint, encoding="utf-8-sig")

print("✅ Consolidação concluída.")
print("• Máscara:", ARQ_MASK)
print("• Baseline DF:", ARQ_DF_BL)
print("• Off-baseline DF:", ARQ_DF_OFF)
print("• Stats baseline:", ARQ_STATS_BL)
print("• Stats off-baseline:", ARQ_STATS_OFF)
print("• Comparativo:", ARQ_COMP)
print("• Checkpoint MD:", ARQ_CHECK_MD)


✅ Consolidação concluída.
• Máscara: C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\outputs\baseline_mask.csv
• Baseline DF: C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\outputs\baseline_datasets\physics_baseline_proxies.csv
• Off-baseline DF: C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\outputs\baseline_datasets\physics_offbaseline_proxies.csv
• Stats baseline: C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\outputs\baseline_datasets\stats_baseline.csv
• Stats off-baseline: C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\outputs\baseline_datasets\stats_offbaseline.csv
• Comparativo: C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\outputs\baseline_datasets\compare_baseline_vs_off.csv
• Checkpoint MD: C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\outputs\CHECKPOINT_BASELINE_FISICO.md


PLANO DE VERIFICAÇÃO (V3) — Passo 2: Construir as Secundárias Canônicas + Primárias e Salvar DF Canônico

     Ler, do seu disco local (Windows), os CSVs curados do projeto.

     Localizar e combinar as colunas‑fonte reais para formar:
     tau_densa, tau_diluida, tau_backpass, delta_proxy, o2_excess_pct (+ primárias).

     Não inventar dados: só combinar medições existentes. Se faltar algo ou houver ambiguidade, parar com mensagem clara pedindo o nome exato da coluna.

     Persistir tudo num DataFrame canônico com nome fixo e autoexplicativo (para nunca esquecer):
     A1_SECONDARIES_FOR_PEDRA_MODEL.csv
          (ficará em ... \outputs\pedra\A1_SECONDARIES_FOR_PEDRA_MODEL.csv)

In [4]:
# ============================================================
# A1 — SEGUNDÁRIAS (FORMA DO NOTEBOOK, SEM INVENTAR)
# Saída: outputs\pedra\A1_SECONDARIES_FOR_PEDRA_MODEL.csv
# Protocolo: V2 + ULTRA-HF-000 (rígido; sem criar colunas ausentes)
# ============================================================

import pandas as pd
from pathlib import Path

# ---------------------------
# CAMINHOS
# ---------------------------
BASE = Path(r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL")
SRC  = BASE / r"data\curated\a1_physics_informed_proxies.csv"
OUT  = BASE / r"outputs\pedra\A1_SECONDARIES_FOR_PEDRA_MODEL.csv"
OUT.parent.mkdir(parents=True, exist_ok=True)

# ---------------------------
# LEITURA (sem heurísticas)
# ---------------------------
try:
    df = pd.read_csv(SRC, engine="c", encoding="utf-8")
except Exception:
    df = pd.read_csv(SRC, sep=None, engine="python", encoding="utf-8")

# Garantias mínimas
if "Timestamp" not in df.columns:
    raise KeyError("[INFORMAÇÃO AUSENTE – PRECISAR PREENCHER] 'Timestamp' não está no CSV de proxies.")
df["Timestamp"] = pd.to_datetime(df["Timestamp"], errors="coerce")
if df["Timestamp"].isna().all():
    raise ValueError("[INFORMAÇÃO AUSENTE – PRECISAR PREENCHER] 'Timestamp' não converteu para datetime no CSV de proxies.")

# ---------------------------
# PRIMÁRIAS (usar apenas se EXISTEM — sem fabricar)
# ---------------------------
flow_col = None
if "coal_flow_furnace_t_h" in df.columns:
    flow_col = "coal_flow_furnace_t_h"
elif "flw_total_c_t_h" in df.columns:
    flow_col = "flw_total_c_t_h"
else:
    raise KeyError("[INFORMAÇÃO AUSENTE – PRECISAR PREENCHER] Não há flow do carvão ('coal_flow_furnace_t_h' nem 'flw_total_c_t_h').")

primarias = [flow_col]
for c in ["total_air_flow_knm3_h", "total_paf_air_flow_knm3_h", "te_of_hot_pri_air_in_aph_outl_adegc"]:
    if c not in df.columns:
        raise KeyError(f"[INFORMAÇÃO AUSENTE – PRECISAR PREENCHER] Variável primária ausente no CSV de proxies: {c}")
    primarias.append(c)

# ---------------------------
# SECUNDÁRIAS (somente as que JÁ existem, como no notebook)
# ---------------------------
secundarias_fixas = ["tau_densa", "tau_diluida", "tau_backpass", "o2_excess_pct"]
faltantes = [c for c in secundarias_fixas if c not in df.columns]
if faltantes:
    raise KeyError(f"[INFORMAÇÃO AUSENTE – PRECISAR PREENCHER] Secundárias ausentes no CSV de proxies: {faltantes}")

# delta_proxy é OPCIONAL no notebook — incluir apenas se existir
secundarias = secundarias_fixas.copy()
if "delta_proxy" in df.columns:
    secundarias.append("delta_proxy")

# ---------------------------
# ORDENAR E SALVAR TABELA CANÔNICA (sem alterações de valor)
# ---------------------------
cols_out = ["Timestamp"] + primarias + secundarias
df_out = df[cols_out].copy()

df_out.to_csv(OUT, index=False, encoding="utf-8")
print(f"[OK] Tabela canônica salva: {OUT}")
print(f"- Flow do carvão usado: {flow_col}")
print(f"- Secundárias incluídas: {', '.join([c for c in secundarias])}")
print(f"- Linhas: {len(df_out)}  |  Colunas: {len(df_out.columns)}")


[OK] Tabela canônica salva: C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\outputs\pedra\A1_SECONDARIES_FOR_PEDRA_MODEL.csv
- Flow do carvão usado: coal_flow_furnace_t_h
- Secundárias incluídas: tau_densa, tau_diluida, tau_backpass, o2_excess_pct
- Linhas: 11757  |  Colunas: 9


Ler o arquivo enriquecido (a1_physics_informed_enriched.csv) que já contém as variáveis primárias e secundárias.

Selecionar as variáveis de pressão candidatas a proxy (conforme a lista no documento “NOME DAS VARIAVEIS E SEUS SIGNIFICADOS.md” e discutido no “Base Teórica, Adequações…”).

Comparar cada proxy de pressão com as séries já calculadas de tau_densa, tau_diluida e tau_backpass, avaliando correlação e consistência de forma (trend) para escolher o(s) melhor(es).

Calcular delta_proxy usando o(s) proxy(s) escolhido(s) e a mesma lógica aplicada aos outros deltas no pipeline físico.

Gerar a série delta_proxy_ref usando a janela temporal do baseline (mesma que wr_ref e wm_ref serão extraídos).

Adicionar essas novas colunas no dataset final que já guarda as secundárias, garantindo que serão usadas no cálculo de wr e wm.

Salvar o dataset atualizado com um nome claro (ex.: A1_SECONDARIES_FOR_PEDRA_MODEL.csv).

In [7]:
# ============================================================
# A1 — TRIAGEM DE PRESSÕES → DELTA_PROXY + REFERÊNCIA BASELINE
# Autor: consultor
# Descrição:
#   1) Lê a base canônica de secundárias (τ e vazões) e o curado (pressões).
#   2) Faz triagem objetiva das colunas de pressão para servir de delta_proxy.
#   3) Calcula delta_proxy (mediana de z-scores das top-K colunas).
#   4) Extrai delta_proxy_ref na MESMA janela de baseline.
#   5) Atualiza o arquivo canônico com a nova coluna e salva diagnósticos.
# Requisitos: pandas, numpy (sem scipy).
# ============================================================

import pandas as pd
import numpy as np
from pathlib import Path
import re

# ---------------------------
# CAMINHOS (ajuste a raiz se necessário)
# ---------------------------
BASE   = Path(r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL")
SEC    = BASE / r"outputs\pedra\A1_SECONDARIES_FOR_PEDRA_MODEL.csv"  # τ + primárias + o2 (já gerado)
CURADO = BASE / r"data\curated\a1_physics_informed.csv"              # pressões + status_operacao
MASK   = BASE / r"outputs\baseline_mask.csv"                         # flag baseline

OUT_DIAG  = BASE / r"outputs\pedra\DELTA_PROXY_DIAGNOSTICS.csv"
OUT_REFS  = BASE / r"outputs\pedra\A1_SECONDARIES_REFS.csv"
OUT_FINAL = SEC  # atualiza o mesmo arquivo canônico

# ---------------------------
# PARÂMETROS
# ---------------------------
TOP_K = 3  # nº de colunas de pressão a combinar (mediana de z-scores)
MIN_PAIRS = 30  # mínimo de pares não-NaN para calcular correlação
NAN_RATE_PENALTY_START = 0.20  # penaliza score acima desse NaN-rate
SMOOTH_SPAN = 25  # suavização exponencial para comparação de tendência (opcional)

# Lista explícita (prioridade física). Usamos SE existir, senão tentamos autodescoberta.
PRESS_PRIORITY_LIST = [
    # Grupo 1 — leito/inferior
    "pressure_1_of_furnace_a_bed_kpa",
    "pressure_1_of_furnace_b_bed_kpa",
    "press_2_3_of_furnace_b_bed_kpa",
    "pressure_3_of_furnace_bed_kpa",
    "pressure_of_furnace_a_lower_kpa",
    "pressure_of_furnace_b_lower_kpa",
    # Grupo 2 — diferenciais de fornalha
    "differential_prs_of_furnace_a_kpa",
    "differential_prs_of_furnace_b_kpa",
    # Grupo 3 — outras pressões de fornalha
    "furnace_pres_pa",
    "pressure_1_of_furnace_a_upper_pa",
    "pressure_1_of_furnace_b_upper_pa",
    "pressure_of_furnace_a_medium_pa",
    "pressure_of_furnace_b_medium_pa",
]

# ---------------------------
# HELPERS
# ---------------------------
def read_csv_smart(path: Path) -> pd.DataFrame:
    try:
        df = pd.read_csv(path, engine="c", encoding="utf-8")
    except Exception:
        df = pd.read_csv(path, sep=None, engine="python", encoding="utf-8")
    drop = [c for c in df.columns if str(c).lower().startswith("unnamed")]
    if drop: df = df.drop(columns=drop)
    if "Timestamp" not in df.columns:
        raise KeyError(f"[ERRO] Sem 'Timestamp' em {path}")
    df["Timestamp"] = pd.to_datetime(df["Timestamp"], errors="coerce")
    if df["Timestamp"].isna().all():
        raise ValueError(f"[ERRO] 'Timestamp' não converteu em {path}")
    return df

def zscore(s: pd.Series) -> pd.Series:
    s = pd.to_numeric(s, errors="coerce")
    mu = s.mean(skipna=True)
    sd = s.std(skipna=True)
    if sd == 0 or np.isnan(sd): 
        return pd.Series(np.nan, index=s.index)
    return (s - mu) / sd

def spearman_safe(a: pd.Series, b: pd.Series, min_pairs=MIN_PAIRS) -> float:
    x = pd.to_numeric(a, errors="coerce")
    y = pd.to_numeric(b, errors="coerce")
    m = x.notna() & y.notna()
    if m.sum() < min_pairs:
        return np.nan
    xr = x[m].rank(method="average")
    yr = y[m].rank(method="average")
    return xr.corr(yr)

def smooth(s: pd.Series, span=SMOOTH_SPAN) -> pd.Series:
    s = pd.to_numeric(s, errors="coerce")
    return s.ewm(span=span, min_periods=max(5, span//5), adjust=False).mean()

def discover_pressure_columns(df: pd.DataFrame) -> list[str]:
    pats = [r"press", r"pressure", r"\bdp\b", r"diff.*press", r"furnace.*press", r"bed.*press"]
    rx = [re.compile(p, re.I) for p in pats]
    cols = []
    for c in df.columns:
        lc = c.lower()
        if lc == "timestamp": 
            continue
        if any(r.search(lc) for r in rx):
            cols.append(c)
    return sorted(set(cols))

# ---------------------------
# 1) Carregar dados
# ---------------------------
sec    = read_csv_smart(SEC)     # τ + primárias + o2 (e delta_proxy se já existir)
curado = read_csv_smart(CURADO)  # pressões + status_operacao
mask = read_csv_smart(MASK)

# 1) identificar a coluna de flag
flag_candidates = ["in_baseline", "is_baseline", "baseline", "flag", "mask"]
flag_col = next((c for c in flag_candidates if c in mask.columns), None)
if flag_col is None:
    raise KeyError(
        "[ERRO] baseline_mask precisa ter uma coluna de flag. "
        f"Tentei: {flag_candidates} | Colunas encontradas: {list(mask.columns)}"
    )

# 2) garantir 'Timestamp' e normalizar o flag para booleano
mask = mask[["Timestamp", flag_col]].copy()
mask["Timestamp"] = pd.to_datetime(mask["Timestamp"], errors="coerce")
if mask["Timestamp"].isna().any():
    raise ValueError("[ERRO] baseline_mask: 'Timestamp' contém valores inválidos (não convertidos para datetime).")

# aceita 0/1, true/false, sim/não
mask["is_baseline"] = (
    mask[flag_col]
      .astype(str).str.strip().str.lower()
      .map({
          "1": True, "true": True, "sim": True, "yes": True,
          "0": False, "false": False, "nao": False, "não": False, "no": False
      })
)

# se vier numérico e cair como NaN acima, tenta converter direto:
na_mask = mask["is_baseline"].isna()
if na_mask.any():
    # tenta numérico direto (0/1)
    as_num = pd.to_numeric(mask.loc[na_mask, flag_col], errors="coerce")
    mask.loc[na_mask & (as_num == 1), "is_baseline"] = True
    mask.loc[na_mask & (as_num == 0), "is_baseline"] = False

# default para qualquer resíduo não mapeado
mask["is_baseline"] = mask["is_baseline"].fillna(False)
# ---------------------------
# 2) Merge e foco em operação
# ---------------------------
# Trazer do curado apenas status_operacao + colunas de pressão
present_from_priority = [c for c in PRESS_PRIORITY_LIST if c in curado.columns]
auto_discovered = discover_pressure_columns(curado)
pressure_cols = sorted(set(present_from_priority + auto_discovered))

if not pressure_cols:
    raise KeyError("[ERRO] Nenhuma coluna de pressão candidata encontrada no curado.")

df = (sec.merge(curado[["Timestamp","status_operacao"] + pressure_cols], on="Timestamp", how="left")
         .merge(mask, on="Timestamp", how="left"))

df = df[df["status_operacao"] == 1].copy()
if df.empty:
    raise ValueError("[ERRO] status_operacao==1 resultou sem linhas após merge.")

df["is_baseline"] = df["is_baseline"].fillna(False)
if df["is_baseline"].sum() == 0:
    raise ValueError("[ERRO] Máscara não marcou baseline dentro do período operado.")

# ---------------------------
# 3) Variáveis de referência para triagem
# ---------------------------
# (usadas para validar aderência física: correlação positiva / tendência similar)
for needed in ["tau_densa","tau_diluida","tau_backpass","total_air_flow_knm3_h","total_paf_air_flow_knm3_h"]:
    if needed not in df.columns:
        raise KeyError(f"[ERRO] coluna necessária ausente no SEC: {needed}")

tau_ref   = df["tau_densa"]  # zona densa como referência primária para cama
v_total   = df["total_air_flow_knm3_h"]
v_primary = df["total_paf_air_flow_knm3_h"]

# Séries suavizadas para checar coerência de tendência (opcional, melhora robustez)
tau_ref_s   = smooth(tau_ref)
v_total_s   = smooth(v_total)
v_primary_s = smooth(v_primary)

# ---------------------------
# 4) Ranking de candidatas
# ---------------------------
rows = []
priority_weight = { # bônus por grupo físico (maior preferência = maior bônus)
    1: 0.10,  # leito/inferior
    2: 0.05,  # diferenciais
    3: 0.00   # demais
}

def group_of(col: str) -> int:
    if col in PRESS_PRIORITY_LIST[:6]: return 1
    if col in PRESS_PRIORITY_LIST[6:8]: return 2
    return 3

for col in pressure_cols:
    s = pd.to_numeric(df[col], errors="coerce")
    nan_rate = s.isna().mean()

    # Correlações rank-based (Spearman) e coerência de tendência (Pearson nas séries suavizadas)
    rho_tau   = spearman_safe(s, tau_ref)
    rho_vtot  = spearman_safe(s, v_total)
    rho_vprim = spearman_safe(s, v_primary)

    pear_tau   = tau_ref_s.corr(smooth(s))          # tendência com τ_densa
    pear_vtot  = v_total_s.corr(smooth(s))          # tendência com v_total
    pear_vprim = v_primary_s.corr(smooth(s))        # tendência com v_primary

    # Corrige sinais "contra-físicos": negativos viram 0 no score
    sp_parts = [rho_tau, rho_vtot, rho_vprim]
    sp_parts = [p if (p is not None and not np.isnan(p) and p > 0) else 0.0 for p in sp_parts]

    tr_parts = [pear_tau, pear_vtot, pear_vprim]
    tr_parts = [p if (p is not None and not np.isnan(p) and p > 0) else 0.0 for p in tr_parts]

    sp_score = float(np.mean(sp_parts))
    tr_score = float(np.mean(tr_parts))

    score = 0.6*sp_score + 0.4*tr_score  # peso maior para Spearman (robusto)
    # Penalização por NaN-rate
    if nan_rate > NAN_RATE_PENALTY_START:
        score *= (1 - min(0.5, (nan_rate - NAN_RATE_PENALTY_START)))  # até -50%

    # Bônus por prioridade física
    grp = group_of(col)
    score += priority_weight[grp]

    rows.append({
        "col": col,
        "group": grp,
        "nan_rate": nan_rate,
        "spearman_tau": rho_tau,
        "spearman_vtotal": rho_vtot,
        "spearman_vprimary": rho_vprim,
        "trend_tau": pear_tau,
        "trend_vtotal": pear_vtot,
        "trend_vprimary": pear_vprim,
        "score": score
    })

diag = pd.DataFrame(rows).sort_values(["group","score"], ascending=[True, False]).reset_index(drop=True)

# ---------------------------
# 5) Escolha top-K e construção do delta_proxy
# ---------------------------
chosen = diag.sort_values(["group","score"], ascending=[True, False]).head(TOP_K)["col"].tolist()
if not chosen:
    raise RuntimeError("[ERRO] Nenhuma coluna elegível para delta_proxy.")

Z = pd.concat([zscore(pd.to_numeric(df[c], errors="coerce")) for c in chosen], axis=1)
df["delta_proxy"] = Z.median(axis=1, skipna=True)

# ---------------------------
# 6) Referência no baseline (mesma janela)
# ---------------------------
bl = df["is_baseline"] == True
delta_proxy_ref = df.loc[bl, "delta_proxy"].median(skipna=True)

# (opcional) também registrar refs úteis das variáveis de apoio
refs = {
    "delta_proxy_ref": delta_proxy_ref,
    "tau_densa_ref":   df.loc[bl, "tau_densa"].median(skipna=True),
    "tau_diluida_ref": df.loc[bl, "tau_diluida"].median(skipna=True),
    "tau_backpass_ref":df.loc[bl, "tau_backpass"].median(skipna=True),
    "v_proxy_total_ref":   v_total[bl].median(skipna=True),
    "v_proxy_primary_ref": v_primary[bl].median(skipna=True),
}
pd.DataFrame([refs]).to_csv(OUT_REFS, index=False, encoding="utf-8")

# ---------------------------
# 7) Salvar diagnósticos e atualizar SEC
# ---------------------------
diag.to_csv(OUT_DIAG, index=False, encoding="utf-8")

# anexar delta_proxy ao arquivo canônico (preserva ordem original + adiciona no fim)
sec_cols = list(sec.columns)
if "delta_proxy" not in sec_cols:
    sec_cols.append("delta_proxy")

sec_updated = sec.merge(df[["Timestamp","delta_proxy"]], on="Timestamp", how="left")
sec_updated = sec_updated[sec_cols]
sec_updated.to_csv(OUT_FINAL, index=False, encoding="utf-8")

print("[OK] delta_proxy criado e salvo em:", OUT_FINAL)
print("[OK] Referências salvas em:", OUT_REFS)
print("[OK] Diagnóstico (ranking) salvo em:", OUT_DIAG)
print(f"[INFO] Colunas escolhidas (TOP_K={TOP_K}): {chosen}")


[OK] delta_proxy criado e salvo em: C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\outputs\pedra\A1_SECONDARIES_FOR_PEDRA_MODEL.csv
[OK] Referências salvas em: C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\outputs\pedra\A1_SECONDARIES_REFS.csv
[OK] Diagnóstico (ranking) salvo em: C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\outputs\pedra\DELTA_PROXY_DIAGNOSTICS.csv
[INFO] Colunas escolhidas (TOP_K=3): ['pressure_of_furnace_b_lower_kpa', 'pressure_of_furnace_a_lower_kpa', 'pressure_1_of_furnace_a_bed_kpa']
